In [ ]:
#For data manipulation
import pandas as pd
import numpy as np
import math
import copy

#To import parquet file with gas prices
import pyarrow

#For plotting
import plotly.graph_objects as go
import plotly.offline as offline
import plotly.subplots as sp
import plotly.io as pio
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
import seaborn as sns

#For feature preprocessing and for tree-based models
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from collections import Counter
from sklearn.impute import SimpleImputer

#Bayesian optimization (one for torch one for sklearn)
from skopt import BayesSearchCV
from bayes_opt import BayesianOptimization

#For neural networks
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
import torch_optimizer as optim1
import torch.optim as optim
from captum.attr import IntegratedGradients

In [ ]:
#Dataframe with pool contract, pair and volume_usd until a certain timepoint
Uniswap_pools_df = pd.read_csv("/Users/fabioza/Desktop/Master thesis/data/Uniswap_final/pool_Uniswap.csv")
#Dataframe with USD and swap prices
Uniswap_prices_df = pd.read_csv("/Users/fabioza/Desktop/Master thesis/data/Uniswap_final/prices_Uniswap.csv")
#Dataframe with swap volume
Uniswap_volume_df = pd.read_csv("/Users/fabioza/Desktop/Master thesis/data/Uniswap_final/volume_Uniswap.csv")
#Dataframe with gas fees
gas_fees = pd.read_parquet('/Users/fabioza/Desktop/Master thesis/data/Uniswap_final/gas_ts_block.par', engine='pyarrow')
#Dataframe with liquidity pool tokens
Uniswap_pool_tokens_df = pd.read_csv("/Users/fabioza/Desktop/Master thesis/data/Uniswap_final/liquidity_tokens_uniswap.csv")

#Same for Sushiswap
Sushiswap_pools_df = pd.read_csv("/Users/fabioza/Desktop/Master thesis/data/Uniswap_final/pool_Sushiswap.csv")
Sushiswap_prices_df = pd.read_csv("/Users/fabioza/Desktop/Master thesis/data/Uniswap_final/prices_Sushiswap.csv")
Sushiswap_volume_df = pd.read_csv("/Users/fabioza/Desktop/Master thesis/data/Uniswap_final/volume_Sushiswap.csv")
Sushiswap_pool_tokens_df = pd.read_csv("/Users/fabioza/Desktop/Master thesis/data/Uniswap_final/liquidity_tokens_sushiswap.csv")


In [ ]:
#Excluding a few Sushiswap pools that don't have enough data (FARM / WETH, KP3R / WETH, USDC / USDT)
exclude_pools = ["0x69b39b89f9274a16e8a19b78e5eb47a4d91dac9e", "0xaf988aff99d3d0cb870812c325c588d8d8cb7de8", "0xd86a120a06255df8d4e2248ab04d4267e23adfaa"] 

Sushiswap_pools_df = Sushiswap_pools_df[~Sushiswap_pools_df["pool"].isin(exclude_pools)]
Sushiswap_prices_df = Sushiswap_prices_df[~Sushiswap_prices_df["pool"].isin(exclude_pools)]
Sushiswap_volume_df = Sushiswap_volume_df[~Sushiswap_volume_df["pool"].isin(exclude_pools)]
Sushiswap_pool_tokens_df = Sushiswap_pool_tokens_df[~Sushiswap_pool_tokens_df["pool_address"].isin(exclude_pools)]


In [ ]:
#Converting date columns to datetime
Uniswap_pools_df['date'] = pd.to_datetime(Uniswap_pools_df['date'])
Uniswap_prices_df['date'] = pd.to_datetime(Uniswap_prices_df['date'])
Uniswap_volume_df['date'] = pd.to_datetime(Uniswap_volume_df['date'])   
Uniswap_pool_tokens_df['date'] = pd.to_datetime(Uniswap_pool_tokens_df['date'])
Sushiswap_pools_df['date'] = pd.to_datetime(Sushiswap_pools_df['date'])
Sushiswap_prices_df['date'] = pd.to_datetime(Sushiswap_prices_df['date'])
Sushiswap_volume_df['date'] = pd.to_datetime(Sushiswap_volume_df['date'])   
Sushiswap_pool_tokens_df['date'] = pd.to_datetime(Sushiswap_pool_tokens_df['date'])

In [ ]:
#Preprocess Uniswap_pool_tokens_df and Sushiswap_pool_tokens_df
#Remove columns
Uniswap_pool_tokens_df = Uniswap_pool_tokens_df.drop(['Unnamed: 0'], axis=1)
Sushiswap_pool_tokens_df = Sushiswap_pool_tokens_df.drop(['Unnamed: 0'], axis=1)
#Rename columns
Uniswap_pool_tokens_df = Uniswap_pool_tokens_df.rename(columns={"totalSupply": "pool_token_amount", 'pool_address': 'pool'})
Sushiswap_pool_tokens_df = Sushiswap_pool_tokens_df.rename(columns={"totalSupply": "pool_token_amount", 'pool_address': 'pool'})


In [ ]:
#Preprocess Sushiswap_volume_df: Rename columns
Sushiswap_volume_df = Sushiswap_volume_df.rename(columns={"token_it": "token_in", '_col3': 'max_token_pair'})

#Only take last observation for a day (df currently contains hourly observations)
Sushiswap_volume_df = Sushiswap_volume_df.sort_values('hour').groupby(['pool', 'date'], as_index=False).last()
Sushiswap_volume_df = Sushiswap_volume_df.drop('hour', axis=1)

In [ ]:
#Concatenating dfs to get one df
complete_volume_df = pd.concat([Uniswap_volume_df, Sushiswap_volume_df])
complete_pool_tokens_df = pd.concat([Uniswap_pool_tokens_df, Sushiswap_pool_tokens_df])

In [ ]:
#Calculate average daily gas fees
gas_fees['ts'] = pd.to_datetime(gas_fees['ts'])
gas_fees = gas_fees.set_index('ts')
average_daily_gas_fees = gas_fees.resample('D')['avg_gas'].mean()
average_daily_gas_fees = average_daily_gas_fees.reset_index()
average_daily_gas_fees = average_daily_gas_fees.rename(columns={"ts": "date"})

In [ ]:
#Creating a mapping that allows correlating smart contract addresses to liq. pairs
pool_mapping_Uniswap = {
    '0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc': ('USDC', 'WETH'),
    '0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852': ('WETH', 'USDT'),
    '0xa478c2975ab1ea89e8196811f51a7b7ade33eb11': ('DAI', 'WETH'),
    '0xbb2b8038a1640196fbe3e38816f3e67cba72d940': ('WBTC', 'WETH'),
    '0x2fdbadf3c4d5a8666bc06645b8358ab803996e28': ('YFI', 'WETH'),
    '0xd3d2e2692501a5c9ca623199d38826e513033a17': ('UNI', 'WETH'),
    '0xa2107fa5b38d9bbd2c461d6edf11b11a50f6b974': ('LINK', 'WETH'),
    '0xc5be99a02c6857f9eac67bbce58df5572498f40c': ('WETH', 'AMPL'),
    '0x3041cbd36888becc7bbcbc0045e3b1f144466f5f': ('USDC', 'USDT'),
    '0xdfc14d2af169b0d36c4eff567ada9b2e0cae044f': ('AAVE', 'WETH'),
    '0xab3f9bf1d81ddb224a2014e98b238638824bcf20': ('LEND', 'WETH'),
    '0xc2adda861f89bbb333c90c492cb837741916a225': ('MKR', 'WETH'),
    '0x43ae24960e5534731fc831386c07755a2dc33d47': ('SNX', 'WETH'),
    '0x3da1313ae46132a397d90d95b1424a9a7e3e0fce': ('WETH', 'CRV'),
    '0x87febfb3ac5791034fd5ef1a615e9d9627c2665d': ('KP3R', 'WETH'),
    '0x32ce7e48debdccbfe0cd037cc89526e4382cb81b': ('CORE', 'WETH'),
    '0xcffdded873554f362ac02f8fb1f02e5ada10516f': ('COMP', 'WETH'),
    '0xdc98556ce24f007a5ef6dc1ce96322d65832a819': ('PICKLE', 'WETH'),
    '0x56feaccb7f750b997b36a68625c7c596f0b41a58': ('FARM', 'WETH'),
    '0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5': ('DAI', 'USDC'),
    '0xddf9b7a31b32ebaf5c064c80900046c9e5b7c65f': ('CREAM', 'WETH'),
    '0xb20bd5d04be54f870d5c0d3ca85d82b34b836405': ('DAI', 'USDT'),
    '0x88d97d199b9ed37c29d846d00d443de980832a22': ('UMA', 'WETH'),
}

pool_mapping_Sushiswap = {
    '0x397ff1542f962076d0bfe58ea045ffa2d347aca0': ('USDC', 'WETH'),
    '0x06da0fd433c1a5d7a4faa01111c044910a184553': ('WETH', 'USDT'),
    '0xc3d03e4f041fd4cd388c549ee2a29a9e5075882f': ('DAI', 'WETH'),
    '0xceff51756c56ceffca006cd410b03ffc46dd3a58': ('WBTC', 'WETH'),
    '0x088ee5007c98a9677165d78dd2109ae4a3d04d0c': ('YFI', 'WETH'),
    '0xdafd66636e2561b0284edde37e42d192f2844d40': ('UNI', 'WETH'),
    '0xc40d16476380e4037e6b1a2594caf6a6cc8da967': ('LINK', 'WETH'),
    '0xcb2286d9471cc185281c4f763d34a962ed212962': ('WETH', 'AMPL'),
    '0xd86a120a06255df8d4e2248ab04d4267e23adfaa': ('USDC', 'USDT'),
    '0xd75ea151a61d06868e31f8988d28dfe5e9df57b4': ('AAVE', 'WETH'),
    '0xba13afecda9beb75de5c56bbaf696b880a5a50dd': ('MKR', 'WETH'),
    '0xa1d7b2d891e3a1f9ef4bbc5be20630c2feb1c470': ('SNX', 'WETH'),
    '0x58dc5a51fe44589beb22e8ce67720b5bc5378009': ('WETH', 'CRV'),
    '0xaf988aff99d3d0cb870812c325c588d8d8cb7de8': ('KP3R', 'WETH'),
    '0x68c6d02d44e16f1c20088731ab032f849100d70f': ('CORE', 'WETH'),
    '0x31503dcb60119a812fee820bb7042752019f2355': ('COMP', 'WETH'),
    '0x269db91fc3c7fcc275c2e6f22e5552504512811c': ('PICKLE', 'WETH'),
    '0xf169cea51eb51774cf107c88309717dda20be167': ('CREAM', 'WETH'),
    '0x001b6450083e531a5a7bf310bd2c1af4247e23d4': ('UMA', 'WETH'),
}

In [ ]:
#Matching pools to smart contract addresses
Uniswap_pools_df['symbol_1'] = Uniswap_pools_df['pool'].map(lambda x: pool_mapping_Uniswap.get(x, (None, None))[0])
Uniswap_pools_df['symbol_2'] = Uniswap_pools_df['pool'].map(lambda x: pool_mapping_Uniswap.get(x, (None, None))[1])
Sushiswap_pools_df['symbol_1'] = Sushiswap_pools_df['pool'].map(lambda x: pool_mapping_Sushiswap.get(x, (None, None))[0])
Sushiswap_pools_df['symbol_2'] = Sushiswap_pools_df['pool'].map(lambda x: pool_mapping_Sushiswap.get(x, (None, None))[1])


In [ ]:
#Correcting the token amounts for Uniswap pools
pool_list_1 = ["0x2fdbadf3c4d5a8666bc06645b8358ab803996e28", "0x32ce7e48debdccbfe0cd037cc89526e4382cb81b", "0x3da1313ae46132a397d90d95b1424a9a7e3e0fce", "0x43ae24960e5534731fc831386c07755a2dc33d47", "0x56feaccb7f750b997b36a68625c7c596f0b41a58", "0x87febfb3ac5791034fd5ef1a615e9d9627c2665d", "0x88d97d199b9ed37c29d846d00d443de980832a22", "0xa2107fa5b38d9bbd2c461d6edf11b11a50f6b974", "0xa478c2975ab1ea89e8196811f51a7b7ade33eb11", "0xab3f9bf1d81ddb224a2014e98b238638824bcf20", "0xc2adda861f89bbb333c90c492cb837741916a225", "0xcffdded873554f362ac02f8fb1f02e5ada10516f", "0xd3d2e2692501a5c9ca623199d38826e513033a17", "0xdc98556ce24f007a5ef6dc1ce96322d65832a819", "0xddf9b7a31b32ebaf5c064c80900046c9e5b7c65f", "0xdfc14d2af169b0d36c4eff567ada9b2e0cae044f"]

for pool in pool_list_1:
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] / 1e18
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] / 1e18

pool_list_2 = ["0x3041cbd36888becc7bbcbc0045e3b1f144466f5f"]    
    
for pool in pool_list_2:
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] / 1e6
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] / 1e6
    
pool_list_3 = ["0xae461ca67b15dc8dc81ce7615e0320da1a9ab8d5", "0xb20bd5d04be54f870d5c0d3ca85d82b34b836405", "0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852"]    
    
for pool in pool_list_3:
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] / 1e18
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] / 1e6
    
pool_list_4 = ["0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc"]    
    
for pool in pool_list_4:
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] / 1e6
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] / 1e18
    
pool_list_5 = ["0xbb2b8038a1640196fbe3e38816f3e67cba72d940"]    
    
for pool in pool_list_5:
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] / 1e8
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] / 1e18
    
pool_list_6 = ["0xc5be99a02c6857f9eac67bbce58df5572498f40c"]    
    
for pool in pool_list_6:
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'x'] / 1e18
    Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] = Uniswap_pools_df.loc[Uniswap_pools_df["pool"] == pool, 'y'] / 1e9

In [ ]:
#Correcting the token amounts for Sushiswap pools
pool_list_1 = ["0x088ee5007c98a9677165d78dd2109ae4a3d04d0c", "0x68c6d02d44e16f1c20088731ab032f849100d70f", "0x58dc5a51fe44589beb22e8ce67720b5bc5378009", "0xa1d7b2d891e3a1f9ef4bbc5be20630c2feb1c470", "0xaf988aff99d3d0cb870812c325c588d8d8cb7de8", "0x001b6450083e531a5a7bf310bd2c1af4247e23d4", "0xc40d16476380e4037e6b1a2594caf6a6cc8da967", "0xc3d03e4f041fd4cd388c549ee2a29a9e5075882f", "0xba13afecda9beb75de5c56bbaf696b880a5a50dd", "0x31503dcb60119a812fee820bb7042752019f2355", "0xdafd66636e2561b0284edde37e42d192f2844d40", "0x269db91fc3c7fcc275c2e6f22e5552504512811c", "0xf169cea51eb51774cf107c88309717dda20be167", "0xd75ea151a61d06868e31f8988d28dfe5e9df57b4"]

for pool in pool_list_1:
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] / 1e18
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] / 1e18

pool_list_2 = ["0xd86a120a06255df8d4e2248ab04d4267e23adfaa"]    
    
for pool in pool_list_2:
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] / 1e6
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] / 1e6
    
pool_list_3 = ["0x06da0fd433c1a5d7a4faa01111c044910a184553"]    
    
for pool in pool_list_3:
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] / 1e18
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] / 1e6
    
pool_list_4 = ["0x397ff1542f962076d0bfe58ea045ffa2d347aca0"]    
    
for pool in pool_list_4:
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] / 1e6
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] / 1e18
    
pool_list_5 = ["0xceff51756c56ceffca006cd410b03ffc46dd3a58"]    
    
for pool in pool_list_5:
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] / 1e8
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] / 1e18
    
pool_list_6 = ["0xcb2286d9471cc185281c4f763d34a962ed212962"]    
    
for pool in pool_list_6:
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'x'] / 1e18
    Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] = Sushiswap_pools_df.loc[Sushiswap_pools_df["pool"] == pool, 'y'] / 1e9


In [ ]:
#Adding a flag indicating which DEX the data is from and concatenating the two dfs
Uniswap_pools_df['Uniswap'] = '1'
Sushiswap_pools_df['Sushiswap'] = '1'
complete_pools_df = pd.concat([Uniswap_pools_df, Sushiswap_pools_df])
complete_pools_df = complete_pools_df.fillna(0)

In [ ]:
#Calculating swap rates between tokens based on amount of tokens in pool
complete_pools_df["avg_swap_price"] = complete_pools_df["y"] / complete_pools_df["x"]

In [ ]:
#Calculating USD swap rates and rearranging direction of token swap rates
Sushiswap_prices_df = Sushiswap_prices_df.rename(columns={"median_usd_price_token_sold": "avg_usd_price_token_sold", "median_usd_price_token_bought": "avg_usd_price_token_bought", "median_swap_price": "avg_swap_price"})

filtered_df = pd.DataFrame()

def filter_and_append(df, filtered_df, pool_mapping):
    for pool_address, (token_a, token_b) in pool_mapping.items():
        filtered_rows = df[(df['pool'] == pool_address) &
                           ((df['token_bought_symbol'] == token_a) & (df['token_sold_symbol'] == token_b) |
                            (df['token_bought_symbol'] == token_b) & (df['token_sold_symbol'] == token_a))]

        grouped_rows = filtered_rows.groupby('date')

        for date, group in grouped_rows:
            #Check if both directions are present (account for the fact that trades can happen in both directions)
            if len(group) == 2:
                #Keep the preferred direction (based on what was defined in dict above)
                preferred_row = group[(group['token_bought_symbol'] == token_a) & (group['token_sold_symbol'] == token_b)]
            else:
                preferred_row = group

                #Check if direction needs to be switched
                if preferred_row.iloc[0]['token_bought_symbol'] != token_a:
                    temp_token_bought_symbol = preferred_row.loc[:, 'token_bought_symbol'].copy()
                    preferred_row.loc[:, 'token_bought_symbol'] = preferred_row.loc[:, 'token_sold_symbol']
                    preferred_row.loc[:, 'token_sold_symbol'] = temp_token_bought_symbol

                    #Switch avg_usd_price_token_sold and avg_usd_price_token_bought
                    temp_avg_usd_price_token_sold = preferred_row.loc[:, 'avg_usd_price_token_sold'].copy()
                    preferred_row.loc[:, 'avg_usd_price_token_sold'] = preferred_row.loc[:, 'avg_usd_price_token_bought']
                    preferred_row.loc[:, 'avg_usd_price_token_bought'] = temp_avg_usd_price_token_sold

                    #Calculate avg_swap_price in opposite direction
                    preferred_row.loc[:, 'avg_swap_price'] = 1 / preferred_row.loc[:, 'avg_swap_price']
            filtered_df = pd.concat([filtered_df, preferred_row], ignore_index=True)
    return filtered_df

filtered_df = filter_and_append(Uniswap_prices_df, filtered_df, pool_mapping_Uniswap)
filtered_df = filter_and_append(Sushiswap_prices_df, filtered_df, pool_mapping_Sushiswap)
filtered_df.reset_index(drop=True, inplace=True)
filtered_df[['avg_usd_price_token_sold', 'avg_usd_price_token_bought', 'avg_swap_price']] = filtered_df[['avg_usd_price_token_sold', 'avg_usd_price_token_bought', 'avg_swap_price']].astype(float)

In [ ]:
#Function to replace extreme outliers with the last non-outlier price: Using a rolling-window approach given the outlier nature of the data
#Window size is number of day taken into account and threshold determines how many standard deviations a value has to deviate to be considered an outlier
def remove_rolling_outliers(df, pool_name, window_size=30, threshold=1.5):
    pool_df = df[df['pool'] == pool_name].sort_values('date').reset_index(drop=True)

    #Rolling mean and stdev calculation
    pool_df['rolling_mean'] = pool_df['avg_swap_price'].rolling(window=window_size, center=True, min_periods=1).mean()
    pool_df['rolling_std'] = pool_df['avg_swap_price'].rolling(window=window_size, center=True, min_periods=1).std()

    #Identification of outliers
    outliers = (pool_df['avg_swap_price'] < (pool_df['rolling_mean'] - threshold * pool_df['rolling_std'])) | \
               (pool_df['avg_swap_price'] > (pool_df['rolling_mean'] + threshold * pool_df['rolling_std']))

    #Iterating through the rows and updating values if current row is outlier
    for index, row in pool_df.loc[outliers].iterrows():
        outlier_date = row['date']

        #Updating outlier value with last non-outlier value
        last_valid_day = pool_df.loc[(pool_df['date'] < outlier_date) & ~outliers].sort_values('date', ascending=False).head(1)
        if not last_valid_day.empty:
            last_values = last_valid_day[['avg_swap_price', 'avg_usd_price_token_sold', 'avg_usd_price_token_bought']].values[0]
            pool_df.loc[index, ['avg_swap_price', 'avg_usd_price_token_sold', 'avg_usd_price_token_bought']] = last_values

    return pool_df.drop(columns=['rolling_mean', 'rolling_std'])

#Removing the outliers
pool_names = filtered_df['pool'].unique()
processed_data = pd.concat([remove_rolling_outliers(filtered_df, pool) for pool in pool_names]).reset_index(drop=True)


In [ ]:
#Visualizing beginning of month prices to check whether they look fine
#Creating a new column to store DEX name
processed_data["platform"] = np.nan
for pool in processed_data["pool"].unique():
    if pool in pool_mapping_Uniswap:
        processed_data.loc[processed_data["pool"] == pool, "platform"] = "Uniswap"
    elif pool in pool_mapping_Sushiswap:
        processed_data.loc[processed_data["pool"] == pool, "platform"] = "Sushiswap"

processed_data['date'] = pd.to_datetime(processed_data['date'])
processed_data.set_index('date', inplace=True)

#Separating Uniswap and Sushiswap data
processed_data_Uniswap = processed_data[processed_data["platform"] == "Uniswap"]
processed_data_Sushiswap = processed_data[processed_data["platform"] == "Sushiswap"]

#Function to process data
def process_data(processed_data):
    grouped_df = processed_data.groupby("pool")
    bom_df = pd.DataFrame()

    #Resampling the data to get beginning of month data
    for pool, group_df in grouped_df:
        
        pool_bom_df = group_df.resample("MS").last()
        bom_df = pd.concat([bom_df, pool_bom_df], ignore_index=False)
    
    #Interpolating nan values
    bom_df.interpolate(method="linear", inplace=True, limit_direction="both")

    #Storing beginning of month prices for each unique token
    unique_tokens = pd.concat([bom_df["token_bought_symbol"], bom_df["token_sold_symbol"]]).unique()
    bom_usd_prices = pd.DataFrame(index=bom_df.index.unique(), columns=unique_tokens)
    for token in unique_tokens:
        token_rows = bom_df[(bom_df["token_bought_symbol"] == token) | (bom_df["token_sold_symbol"] == token)]
        token_rows = token_rows.loc[~token_rows.index.duplicated(keep='first')]
        bom_usd_prices[token] = token_rows.apply(lambda row: row["avg_usd_price_token_bought"] if row["token_bought_symbol"] == token else row["avg_usd_price_token_sold"], axis=1)
    
    #Interpolating usd prices that are still missing
    bom_usd_prices.interpolate(method="linear", inplace=True, limit_direction="both")
    return bom_usd_prices

#Apply function to Uniswap and Sushiswap data separately
bom_usd_prices_Uniswap = process_data(processed_data_Uniswap)
bom_usd_prices_Sushiswap = process_data(processed_data_Sushiswap)

fig, ax = plt.subplots(2, 1, sharex=True)
bom_usd_prices_Uniswap.plot(ax=ax[0])
ax[0].set_ylabel("USD Price")
ax[0].set_title("USD Price at the Beginning of Each Month for Unique Tokens on Uniswap")
ax[0].legend(title="Tokens")
bom_usd_prices_Sushiswap.plot(ax=ax[1])
ax[1].set_ylabel("USD Price")
ax[1].set_xlabel("Date")
ax[1].set_title("USD Price at the Beginning of Each Month for Unique Tokens on Sushiswap")
ax[1].legend(title="Tokens")
plt.tight_layout()
plt.show()


In [ ]:
#Merging all the dataframes
complete_pools_df = pd.merge(complete_pools_df, complete_volume_df, on=['pool', 'date'])

#Removing unnecessary columns
complete_pools_df = complete_pools_df.drop(columns = ["max_token_pair", "token_in", "token_out"])

#Calculating order flow imbalance
complete_pools_df["imbalance"] = complete_pools_df["volume_in"] - complete_pools_df["volume_out"]

#Adding gas fees to dataframe
complete_pools_df = pd.merge(complete_pools_df, average_daily_gas_fees, on=['date'])

#Adding liquidity tokens to dataframe
complete_pools_df = pd.merge(complete_pools_df, complete_pool_tokens_df, on=['pool', 'date'])

In [ ]:
#Calculating the total pool value (=pool size) for each day
def calculate_total_pool_value(prices_df, pools_df):
    prices_df_reversed = prices_df.copy()
    prices_df_reversed.rename(columns={
        'token_bought_symbol': 'token_sold_symbol', 
        'token_sold_symbol': 'token_bought_symbol',
        'avg_usd_price_token_bought': 'avg_usd_price_token_sold',
        'avg_usd_price_token_sold': 'avg_usd_price_token_bought'
    }, inplace=True)

    prices_df_combined = pd.concat([prices_df, prices_df_reversed])

    merged_df = pd.merge(pools_df, prices_df_combined, 
                         left_on=['date', 'symbol_1', 'symbol_2'], 
                         right_on=['date', 'token_bought_symbol', 'token_sold_symbol'], 
                         how='left')

    merged_df['USD_value_symbol_1'] = merged_df['x'] * merged_df['avg_usd_price_token_bought']
    merged_df['USD_value_symbol_2'] = merged_df['y'] * merged_df['avg_usd_price_token_sold']
    merged_df['total_USD_value'] = merged_df['USD_value_symbol_1'] + merged_df['USD_value_symbol_2']
    merged_df = merged_df.rename({'pool_x': 'pool'}, axis=1)
    merged_df = merged_df[['pool', 'date', 'total_USD_value']]
    merged_df = merged_df.drop_duplicates(subset=['pool', 'date'])
    return merged_df

#Calculate for Uniswap
Uniswap_merged_df = calculate_total_pool_value(Uniswap_prices_df, Uniswap_pools_df)

#Calculate for Sushiswap
Sushiswap_merged_df = calculate_total_pool_value(Sushiswap_prices_df, Sushiswap_pools_df)

#Concatenate both dataframes
final_df = pd.concat([Uniswap_merged_df, Sushiswap_merged_df])

In [ ]:
#Merging pool size with the complete dataframe
complete_pools_df = pd.merge(final_df, complete_pools_df, on=['pool', 'date'], how='inner')

In [ ]:
#Adding a categorical variable that declares the pool type
#Clearly specifying exotic and stable pairs and adding rest to normal pairs
pair_categories = {
    'WETH-AMPL': 'Exotic',
    'USDC-USDT': 'Stable',
    'DAI-USDC': 'Stable',
    'DAI-USDT': 'Stable',
    'PICKLE-WETH': 'Exotic',
    'FARM-WETH': 'Exotic',
    'CREAM-WETH': 'Exotic',
     'CORE-WETH': 'Exotic'
}

def assign_category(row):
    pair = row['symbol_1'] + '-' + row['symbol_2']
    return pair_categories.get(pair, 'Normal')

complete_pools_df['pool_category'] = complete_pools_df.apply(assign_category, axis=1)

In [ ]:
#Converting DEX flags to dtype int
complete_pools_df['Uniswap'] = complete_pools_df['Uniswap'].astype('int')
complete_pools_df['Sushiswap'] = complete_pools_df['Sushiswap'].astype('int')

In [ ]:
#Calculating profits of liquidity provision on both dex across cross-section of pairs
bom_usd_prices_dataframes = {
    "Uniswap": bom_usd_prices_Uniswap, 
    "Sushiswap": bom_usd_prices_Sushiswap
}

complete_pools_Uniswap_df = complete_pools_df[complete_pools_df['Uniswap'] == 1]
complete_pools_Sushiswap_df = complete_pools_df[complete_pools_df['Sushiswap'] == 1]

complete_pools_Uniswap_df.set_index('date', inplace=True)
complete_pools_Sushiswap_df.set_index('date', inplace=True)
complete_pools_df = complete_pools_df.set_index("date")

#How much of the pools total liquidity to be invested
investment_percentages = [0.01]  

#timeframe of observation (equals ten three month periods)
start_date = pd.to_datetime('2020-10-01')
end_date = pd.to_datetime('2023-04-01')

#Calculating profit for each DEX
profit_results = []

for platform, complete_pools_df in [("Uniswap", complete_pools_Uniswap_df), ("Sushiswap", complete_pools_Sushiswap_df)]:
    bom_usd_prices = bom_usd_prices_dataframes[platform]

    start_date = pd.to_datetime('2020-10-01')
    end_date = pd.to_datetime('2023-04-01')

    while start_date < end_date:
        next_date = start_date + pd.DateOffset(months=1)

        for pool in complete_pools_df['pool'].unique():
            pool_data = complete_pools_df[complete_pools_df['pool'] == pool]

            pool_data_monthly = pool_data.resample('MS').first()

            token_bought_symbol = pool_data['symbol_1'].iloc[0]
            token_sold_symbol = pool_data['symbol_2'].iloc[0]

            bom_prices_token_bought = bom_usd_prices[token_bought_symbol]
            bom_prices_token_sold = bom_usd_prices[token_sold_symbol]

            #Calculating USD pool value of each token in a pool at timepoint t_1 and t_2
            pool_value_t1_token_bought = pool_data_monthly.loc[start_date, 'x'] * bom_prices_token_bought.loc[start_date]
            pool_value_t1_token_sold = pool_data_monthly.loc[start_date, 'y'] * bom_prices_token_sold.loc[start_date]
            pool_value_t2_token_bought = pool_data_monthly.loc[next_date, 'x'] * bom_prices_token_bought.loc[next_date]
            pool_value_t2_token_sold = pool_data_monthly.loc[next_date, 'y'] * bom_prices_token_sold.loc[next_date]

            #Calculating the total pool value by summing both
            total_pool_value_t1 = pool_value_t1_token_bought + pool_value_t1_token_sold
            total_pool_value_t2 = pool_value_t2_token_bought + pool_value_t2_token_sold

            #Getting amount of pool tokens at t_1 and t_2
            lp_tokens_t1 = pool_data_monthly.loc[start_date, 'pool_token_amount']
            lp_tokens_t2 = pool_data_monthly.loc[next_date, 'pool_token_amount']
            
            for investment_percentage in investment_percentages:
                #Calculating how much is invested in terms of USD at t_1
                invest_t1 = investment_percentage * total_pool_value_t1
                #Calculating the amount of liquidity tokens a liq. providers gets in t_1 based on the investment amount
                amount0 = investment_percentage * pool_data_monthly.loc[start_date, 'x']
                amount1 = investment_percentage * pool_data_monthly.loc[start_date, 'y']
                _totalSupply = pool_data_monthly.loc[start_date, 'pool_token_amount']
                _reserve0 = pool_data_monthly.loc[start_date, 'x']
                _reserve1 = pool_data_monthly.loc[start_date, 'y']
                liquidity_t1 = min(amount0 * _totalSupply / _reserve0, amount1 * _totalSupply / _reserve1)

                #Calculating how much (both absolute and relative) of the liquidity tokens the same LP owns in t_2
                lp_tokens_owned = liquidity_t1
                percentage_t2 = lp_tokens_owned / lp_tokens_t2
                #Based on this calculating the USD value of his liq. position
                invest_t2 = percentage_t2 * total_pool_value_t2
                #Calculating the return of just holding the tokens
                hold_t1 = invest_t1 * (bom_prices_token_bought.loc[start_date] + bom_prices_token_sold.loc[start_date])
                hold_t2 = invest_t2 * (bom_prices_token_bought.loc[next_date] + bom_prices_token_sold.loc[next_date])
                
                #Calculating the return of providing liquidity vs holding the token
                return_value = 100 * ((invest_t2 / hold_t1) - (invest_t1 / hold_t1)) / (invest_t1 / hold_t1)

                #Appending returns and other variables to dict
                profit_results.append({
                    'platform': platform,
                    'pool': pool,
                    'start_date': start_date,
                    'end_date': next_date,
                    'return': return_value
                })

        start_date = next_date

#Converting the dict to a dataframe
profit_df = pd.DataFrame(profit_results)
print(profit_df)


In [ ]:
#Concatenating the two pool dfs together again, as they get split in previous codeblock
complete_pools_df = pd.concat([complete_pools_Uniswap_df, complete_pools_Sushiswap_df])

In [ ]:
#Adding token symbols to contract addresses (for identification)
def get_token_symbols(pool_address):
    return pool_mapping_Uniswap.get(pool_address, ('Unknown', 'Unknown'))
profit_df['symbol_1'], profit_df['symbol_2'] = zip(*profit_df['pool'].map(get_token_symbols))

In [ ]:
#Plotting profits for both pools
#Splitting data by DEX
profit_df_Uniswap = profit_df[profit_df['platform'] == 'Uniswap']
profit_df_Sushiswap = profit_df[profit_df['platform'] == 'Sushiswap']

#Uniswap plot
traces_Uniswap = []
for pool in profit_df_Uniswap['pool'].unique():
    pool_data = profit_df_Uniswap[profit_df_Uniswap['pool'] == pool]
    trace = go.Scatter(
        x=pool_data['start_date'],
        y=pool_data['return'],
        mode='lines',
        name=f'{pool} Pool'
    )
    traces_Uniswap.append(trace)

layout_Uniswap = go.Layout(
    title='Uniswap one-month pool profit over time',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Profit (%)')
)

fig_Uniswap = go.Figure(data=traces_Uniswap, layout=layout_Uniswap)

#Sushiswap plot
traces_Sushiswap = []
for pool in profit_df_Sushiswap['pool'].unique():
    pool_data = profit_df_Sushiswap[profit_df_Sushiswap['pool'] == pool]
    trace = go.Scatter(
        x=pool_data['start_date'],
        y=pool_data['return'],
        mode='lines',
        name=f'{pool} Pool'
    )
    traces_Sushiswap.append(trace)

layout_Sushiswap = go.Layout(
    title='Sushiswap one-month pool profit over time',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Profit (%)')
)

fig_Sushiswap = go.Figure(data=traces_Sushiswap, layout=layout_Sushiswap)

offline.init_notebook_mode(connected=True)
offline.plot(fig_Uniswap, filename='Uniswap_profit_plot.html', auto_open=False)
offline.plot(fig_Sushiswap, filename='Sushiswap_profit_plot.html', auto_open=False)
offline.iplot(fig_Uniswap)
offline.iplot(fig_Sushiswap)

In [ ]:
#Same but next to each other

traces_Uniswap = []
for pool in profit_df_Uniswap['pool'].unique():
    pool_data = profit_df_Uniswap[profit_df_Uniswap['pool'] == pool]
    trace = go.Scatter(
        x=pool_data['start_date'],
        y=pool_data['return'],
        mode='lines',
        showlegend=False
    )
    traces_Uniswap.append(trace)

traces_Sushiswap = []
for pool in profit_df_Sushiswap['pool'].unique():
    pool_data = profit_df_Sushiswap[profit_df_Sushiswap['pool'] == pool]
    trace = go.Scatter(
        x=pool_data['start_date'],
        y=pool_data['return'],
        mode='lines',
        showlegend=False
    )
    traces_Sushiswap.append(trace)

fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('Uniswap', 'Sushiswap'))

for trace in traces_Uniswap:
    fig.add_trace(trace, row=1, col=1)

for trace in traces_Sushiswap:
    fig.add_trace(trace, row=1, col=2)

fig.update_layout(height=600, width=1200, title_text="One-month pool returns over analysis timeframe")

fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Return (%)", row=1, col=1)

fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_yaxes(title_text="Return (%)", row=1, col=2)

offline.init_notebook_mode(connected=True)
offline.plot(fig, filename='combined_profit_plot.html', auto_open=False)
offline.iplot(fig)

pio.write_image(fig, '/Users/fabioza/Desktop/Master thesis/sushi_uni_1m_returns.png')


In [ ]:
complete_pools_df = complete_pools_df.reset_index()

#Calculating daily returns, based on returns the 30d vola and impermanent loss for each pair in the liquidity pool
complete_pools_df['date'] = pd.to_datetime(complete_pools_df['date'])
complete_pools_df = complete_pools_df.sort_values(['pool', 'date'])
complete_pools_df['daily_return'] = complete_pools_df.groupby('pool')['avg_swap_price'].pct_change()

#Vola as 30d stdev of daily returns
complete_pools_df['30D_volatility'] = complete_pools_df.groupby('pool')['daily_return'].rolling(window=30).std().reset_index(0, drop=True)

#impermanent loss as the percentage change of the ratio between the two tokens in the pool btw. time t_1 and t_2 
def impermanent_loss(pt1, pt2):
    ratio = pt2 / pt1
    return 100 * ((2 * np.sqrt(ratio)) / (1 + ratio) - 1)

complete_pools_df['pt1'] = complete_pools_df.groupby('pool')['avg_swap_price'].shift(29)
complete_pools_df['30D_impermanent_loss'] = complete_pools_df.apply(lambda x: impermanent_loss(x['pt1'], x['avg_swap_price']) if pd.notnull(x['pt1']) else np.nan, axis=1)

complete_pools_df = complete_pools_df.drop(columns=['pt1'])
complete_pools_df = complete_pools_df.drop(columns=['daily_return'])


In [ ]:
complete_pools_df = complete_pools_df.set_index('date')

#Resampling and aggregating features on a 1-month horizon
df_resampled = complete_pools_df.groupby('pool').resample('1M').agg({
    'volume_usd': 'sum',
    'imbalance': 'mean',
    'avg_gas': 'mean',
    '30D_volatility': 'last',
    '30D_impermanent_loss': 'last',
    'pool_category': 'first',
    'total_USD_value': 'first',
    'Uniswap': 'first',
    'Sushiswap': 'first'
})

df_resampled.reset_index(inplace=True)

#Making sure that date capture the first day of the month
df_resampled['date'] = df_resampled['date'] + pd.offsets.MonthBegin(-1)

df_resampled.rename(columns={'date': 'start_date'}, inplace=True)

#Adding profits (the feature we want to predict)
df_final = pd.merge(df_resampled, profit_df, on=['pool', 'start_date'])

#As we want to predict the return in one month from now we need to introduce this as a feature
df_final['return_in_1month'] = df_final.groupby('pool')['return'].shift(-1)

In [ ]:
#For two observations total_USD_value contains inf, I fill up this observation with the average of the observation before it and after it. This shouldn't bias results
pool_values = ["0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc", "0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852"]

for pool_value in pool_values:
    pool_rows = df_final[df_final["pool"] == pool_value]
    filled_values = pool_rows["total_USD_value"].rolling(3, min_periods=1, center=True).mean()
    df_final.loc[df_final["pool"] == pool_value, "total_USD_value"] = filled_values

In [ ]:
#Sorting data by date and then by pool for tree based models
df_final = df_final.sort_values(by=['start_date', 'pool'])

In [ ]:
#Creating a separate dataframe for recurrent networks
df_final_RNN = df_final.copy()

In [ ]:
#Dropping columns
df_final = df_final.drop(columns=['symbol_1', 'symbol_2', 'platform', 'start_date', 'end_date'])

In [ ]:
#Dropping na values
df_final = df_final.dropna()
df_final_RNN = df_final_RNN.dropna()

## Summary stats

In [ ]:
profit_df_analysis = profit_df.copy()

In [ ]:
#Assigning pool categories to pool contract addresses
profit_df_analysis['pool_category'] = profit_df_analysis.apply(assign_category, axis=1)

#Grouping by pool category and counting positive and negative returns and calculating stdev
results = profit_df_analysis.groupby('pool_category')['return'].agg(
    positive_count = lambda x: (x > 0).sum(),
    negative_count = lambda x: (x < 0).sum(),
    std_dev = 'std'
)

#Calculating ratios per category
results['positive_ratio'] = results['positive_count'] / (results['positive_count'] + results['negative_count'])
results['negative_ratio'] = results['negative_count'] / (results['positive_count'] + results['negative_count'])

print(results)

In [ ]:
#Number of positive vs. negative returns before forecasting / testing period
filtered_df = df_final_RNN[df_final_RNN["start_date"] < "2022-08-01"]

num_positives = filtered_df[filtered_df["return_in_1month"] > 0].shape[0]
num_negatives = filtered_df[filtered_df["return_in_1month"] < 0].shape[0]

print(f"Number of positive returns: {num_positives}")
print(f"Number of negative returns: {num_negatives}")
print(f"Ratio pos.: {num_positives / (num_positives+num_negatives)}")

In [ ]:
#Number of positive vs. negative returns during forecasting / testing period
filtered_df = df_final_RNN[df_final_RNN["start_date"] > "2022-08-01"]

num_positives = filtered_df[filtered_df["return_in_1month"] > 0].shape[0]
num_negatives = filtered_df[filtered_df["return_in_1month"] < 0].shape[0]

print(f"Number of positive returns: {num_positives}")
print(f"Number of negative returns: {num_negatives}")
print(f"Ratio neg. to pos.: {num_negatives}")
print(f"Ratio pos.: {num_positives / (num_positives+num_negatives)}")

In [ ]:
#General descriptive stats
numerical_cols = ['volume_usd', 'imbalance', 'avg_gas', '30D_volatility', '30D_impermanent_loss', 'total_USD_value', 'return', 'return_in_1month']

df_final[numerical_cols].describe()

In [ ]:
#Descriptive stats per pool category
df_final.groupby('pool_category')[numerical_cols].agg(['mean', 'std'])

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x='30D_volatility', y='return', hue='pool_category', data=df_final)
plt.xlabel('30-days volatility') 
plt.ylabel('Return') 
plt.title('Return vs. volatility for different pool types')
plt.show()

## Linear model

In [ ]:
#Resampling and aggregating features on a 3-month horizon
df_resampled = complete_pools_df.groupby('pool').resample('1M').agg({
    'volume_usd': 'sum',
    'imbalance': 'mean',
    'avg_gas': 'mean',
    '30D_volatility': 'last',
    '30D_impermanent_loss': 'last',
    'pool_category': 'first',
    'total_USD_value': 'first',
    'Uniswap': 'first',
    'Sushiswap': 'first'
})

df_resampled.reset_index(inplace=True)

#Making sure that date capture the first day of the month
df_resampled['date'] = df_resampled['date'] + pd.offsets.MonthBegin(-1)

df_resampled.rename(columns={'date': 'start_date'}, inplace=True)

#Adding profits (the feature we want to predict)
df_final_linear = pd.merge(df_resampled, profit_df, on=['pool', 'start_date'])

#As we want to predict the return in three months from now we need to introduce this as a feature
df_final_linear['return_in_1month'] = df_final_linear.groupby('pool')['return'].shift(-1)

#The last observation for each pool is empty as there is no return in three months, this observation has to be dropped
df_final_linear = df_final_linear.dropna(subset=['return_in_1month'])

#Dropping first row for each pool which now has NaN values
df_final_linear.dropna(inplace=True)

In [ ]:
#Further preprocessing the data and getting it into the rigth format
#Encoding "pool" and "pool_category" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder_pool = OneHotEncoder(sparse_output=False)
pool_encoded = encoder_pool.fit_transform(df_final_linear[['pool']])
pool_columns = ['pool_' + str(i) for i in range(pool_encoded.shape[1])]

encoder_pool_category = OneHotEncoder(sparse_output=False)
pool_category_encoded = encoder_pool_category.fit_transform(df_final_linear[['pool_category']])
pool_category_columns = ['pool_category_' + str(i) for i in range(pool_category_encoded.shape[1])]

#Defining the features and the target
df_final_linear = df_final_linear.drop(columns=['pool', 'pool_category'])

#Concatenating one hot-encoded and other features to same dataframe
df_encoded_linear = pd.concat([
    pd.DataFrame(pool_encoded, columns=pool_columns),
    pd.DataFrame(pool_category_encoded, columns=pool_category_columns),
    df_final_linear.reset_index(drop=True)
], axis=1)

#One value in the pool size columns couldn't be calculated, it's replaced with the mean which shouldn't bias results
df_encoded_linear['total_USD_value'].replace([np.inf, -np.inf], np.nan, inplace=True)
col_index = df_encoded_linear.columns.get_loc('total_USD_value')
imputer = SimpleImputer(strategy='mean')
total_usd_value = df_encoded_linear.iloc[:, col_index].values.reshape(-1, 1)
df_encoded_linear.iloc[:, col_index] = imputer.fit_transform(total_usd_value)

In [ ]:
#Dropping unnecessary columns
df_encoded_linear = df_encoded_linear.drop(columns = ["symbol_1", "symbol_2", "end_date", "platform"])

In [ ]:
#Converting df to csv and saving locally
df_encoded_linear.to_csv("/Users/fabioza/Desktop/Master thesis/data/df_encoded_linear30d.csv")

In [ ]:
#Code continued in R

## Random Forest

In [ ]:
#Further preprocessing the data and getting it into the rigth format to avoid leakage due to time-series nature of data
df_no_missing = df_final.dropna()

#Encoding "pool" and "pool_category" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder_pool = OneHotEncoder(sparse_output=False)
pool_encoded = encoder_pool.fit_transform(df_no_missing[['pool']])
pool_columns = ['pool_' + str(i) for i in range(pool_encoded.shape[1])]

encoder_pool_category = OneHotEncoder(sparse_output=False)
pool_category_encoded = encoder_pool_category.fit_transform(df_no_missing[['pool_category']])
pool_category_columns = ['pool_category_' + str(i) for i in range(pool_category_encoded.shape[1])]

#Defining the features and the target
X = df_no_missing.drop(columns=['pool', 'pool_category', 'return_in_1month'])
y = df_no_missing['return_in_1month']

#Concatenating one hot-encoded and other features to same dataframe
df_encoded = pd.concat([
    pd.DataFrame(pool_encoded, columns=pool_columns),
    pd.DataFrame(pool_category_encoded, columns=pool_category_columns),
    X.reset_index(drop=True)
], axis=1)

#One value in the pool size columns couldn't be calculated, it's replaced with the mean which shouldn't bias results
df_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
imputer = SimpleImputer(strategy='mean')
df_encoded_filled = pd.DataFrame(imputer.fit_transform(df_encoded), columns=df_encoded.columns)

#Splitting data into train / validating and test set
#Train and validation sets are used to find the best model, test set can later be used to test the best model
#According to this 4 3-month periods are used for training, 2 for validation and 2 for testing
train_size = 0.8
val_size = 0.1
test_size = 0.1

#Calculate the sizes based on the proportions
num_samples = len(df_encoded_filled)
train_samples = int(train_size * num_samples)
val_samples = int(val_size * num_samples)
test_samples = num_samples - train_samples - val_samples

X_train = df_encoded_filled[:train_samples]
X_val = df_encoded_filled[train_samples:train_samples + val_samples]
X_test = df_encoded_filled[train_samples + val_samples:]
y_train = y[:train_samples]
y_val = y[train_samples:train_samples + val_samples]
y_test = y[train_samples + val_samples:]

#Concatenating train and validation dfs
X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])

#Initializing the random forest model
model = RandomForestRegressor(random_state=63)

#Defining parameter grid (number of trees and depth of each tree)
param_grid = {
    'n_estimators': (10, 1000),  
    'max_depth': (1, 20),
}

#Performing time-series cross-validation to account for time-series anture of data
#75% of the total data is in the train / validation set, this corresponds to six observations per pool (which explains the split)
#Each fold is being used once as the validation set and the rest as the training set
tscv = TimeSeriesSplit(n_splits=6)

#Using bayes optimization to find the best hyperparam combination for my model based on the grid defined earlier
bayes_search = BayesSearchCV(model, param_grid, scoring='neg_mean_absolute_error', cv=tscv, n_jobs=-1, n_iter=50)
bayes_search.fit(X_trainval, y_trainval)

#Finding the best model (maximizing negative MAE (=minimizing MAE))
best_model = bayes_search.best_estimator_

print("Best Parameters:", bayes_search.best_params_)
print("Best Score (Negative MAE):", bayes_search.best_score_)

mse_list = []
rmse_list = []
mae_list = []
hit_rate_list = []

#Expanding window is being used, which means that in the first iteration the first validation fold is being forecasted and
#in the second run, it is being added to the training set and the last validation fold is being forecasted
window_length = 2

#Fitting the previously found best model to the training data and evaluating it on the validation data
for train_index, val_index in tscv.split(X_trainval):
    X_train, X_val = X_trainval.iloc[train_index], X_trainval.iloc[val_index]
    y_train, y_val = y_trainval.iloc[train_index], y_trainval.iloc[val_index]
    if len(X_train) >= window_length:
        best_model.fit(X_train, y_train)
        y_pred_val = best_model.predict(X_val)
        
        #Calculating evaluation metrics on the validation set
        mse_val = mean_squared_error(y_val, y_pred_val)
        mse_list.append(mse_val)
        rmse_val = math.sqrt(mse_val)
        rmse_list.append(rmse_val)
        mae_val = mean_absolute_error(y_val, y_pred_val)
        mae_list.append(mae_val)
        hit_rate_val = np.mean((y_val > 0) == (y_pred_val > 0))
        hit_rate_list.append(hit_rate_val)

print(f"Average MSE on Validation Set: {np.mean(mse_list)}")
print(f"Average RMSE on Validation Set: {np.mean(rmse_list)}")
print(f"Average MAE on Validation Set: {np.mean(mae_list)}")
print(f"Average Hit Rate on Validation Set: {np.mean(hit_rate_list)}")

## Gradient Boosting

In [ ]:
#Model with Gradient Boosting with grid search and bayesian optimization
#Best parameters with hyperparam search are found based on train / validation set
df_no_missing = df_final.dropna()

#Defining target and features
y = df_no_missing['return_in_1month']
X = df_no_missing.drop(columns=['pool', 'pool_category'])

#Encoding "pool" and "pool_category" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder_pool = OneHotEncoder(sparse_output=False)
pool_encoded = encoder_pool.fit_transform(df_no_missing[['pool']])
pool_columns = ['pool_' + str(i) for i in range(pool_encoded.shape[1])]

encoder_pool_category = OneHotEncoder(sparse_output=False)
pool_category_encoded = encoder_pool_category.fit_transform(df_no_missing[['pool_category']])
pool_category_columns = ['pool_category_' + str(i) for i in range(pool_category_encoded.shape[1])]

#Concatenating one hot-encoded and other features to same dataframe
df_encoded = pd.concat([
    pd.DataFrame(pool_encoded, columns=pool_columns),
    pd.DataFrame(pool_category_encoded, columns=pool_category_columns),
    X.reset_index(drop=True)
], axis=1)

#One value in the pool size columns couldn't be calculated, it's replaced with the mean which shouldn't bias results
df_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
df_encoded.fillna(df_encoded.mean(), inplace=True)

X = df_encoded.drop(columns=['return_in_1month'])

#Initializing the gradient boosting model, adding regularization and early stopping to avoid overfitting the model
model = GradientBoostingRegressor(min_samples_split = 10, min_samples_leaf = 4, 
                                  random_state=63, max_depth=3, max_features='sqrt', 
                                  subsample=0.8, n_iter_no_change=5, tol=0.01)

#Defining parameter grid (number of trees, depth of each tree and learning rate)
param_grid = {
    'n_estimators': (1, 100),   
    'max_depth': (1, 5),          
    'learning_rate': (0.001, 0.1)   
}

#Splitting data into train / validating and test set
#Train and validation sets are used to find the best model, test set can later be used to test the best model
train_size = 0.8 
val_size = 0.1
test_size = 0.1

#Calculate the sizes based on the proportions
num_samples = len(df_encoded)
train_samples = int(train_size * num_samples)
val_samples = int(val_size * num_samples)
test_samples = num_samples - train_samples - val_samples

X_train = df_encoded[:train_samples]
X_val = df_encoded[train_samples:train_samples + val_samples]
X_test = df_encoded[train_samples + val_samples:]
y_train = y[:train_samples]
y_val = y[train_samples:train_samples + val_samples]
y_test = y[train_samples + val_samples:]

#Concatenating train and validation dfs
X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])

#Performing time-series cross-validation to account for time-series anture of data
#75% of the total data is in the train / validation set, this corresponds to six observations per pool (which explains the split)
#Each fold is being used once as the validation set and the rest as the training set
tscv = TimeSeriesSplit(n_splits=6)

#Using bayes optimization to find the best hyperparam combination for my model based on the grid defined earlier
bayes_search = BayesSearchCV(model, param_grid, scoring='neg_mean_absolute_error', cv=tscv, n_jobs=-1, n_iter=50)
bayes_search.fit(X_trainval, y_trainval)

#Finding the best model (maximizing negative MAE (=minimizing MAE))
best_model = bayes_search.best_estimator_

print("Best Parameters:", bayes_search.best_params_)
print("Best Score (Negative MAE):", bayes_search.best_score_)

mse_list = []
rmse_list = []
mae_list = []
hit_rate_list = []

#Expanding window is being used, which means that in the first iteration the first validation fold is being forecasted and
#in the second run, it is being added to the training set and the last validation fold is being forecasted
window_length = 2

#Fitting the previously found best model to the training data and evaluating it on the validation data
for train_index, val_index in tscv.split(X_trainval):
    X_train, X_val = X_trainval.iloc[train_index], X_trainval.iloc[val_index]
    y_train, y_val = y_trainval.iloc[train_index], y_trainval.iloc[val_index]
    
    if len(X_train) >= window_length:
        best_model.fit(X_train, y_train)
        y_pred_val = best_model.predict(X_val)

        #Calculating evaluation metrics on the validation set
        mse_val = mean_squared_error(y_val, y_pred_val)
        mse_list.append(mse_val)
        rmse_val = math.sqrt(mse_val)
        rmse_list.append(rmse_val)
        mae_val = mean_absolute_error(y_val, y_pred_val)
        mae_list.append(mae_val)
        hit_rate_val = np.mean((y_val > 0) == (y_pred_val > 0))
        hit_rate_list.append(hit_rate_val)

print(f"Average MSE on Validation Set: {np.mean(mse_list)}")
print(f"Average RMSE on Validation Set: {np.mean(rmse_list)}")
print(f"Average MAE on Validation Set: {np.mean(mae_list)}")
print(f"Average Hit Rate on Validation Set: {np.mean(hit_rate_list)}")

## Deep neural network

In [ ]:
#One with Bayesian Optimization (SGD) and time-series cross-validation
#Encoding "pool" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder = OneHotEncoder(sparse_output=False)
pool_encoded = encoder.fit_transform(df_final[['pool']])
pool_columns = ['pool_' + str(i) for i in range(pool_encoded.shape[1])]

#Concatenating one hot-encoded and other features to same dataframe
df_encoded = pd.concat([pd.DataFrame(pool_encoded, columns=pool_columns), df_final.reset_index(drop=True)], axis=1)

#Encoding "pool_category" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder_pool_category = OneHotEncoder(sparse_output=False)
pool_category_encoded = encoder_pool_category.fit_transform(df_final[['pool_category']])
pool_category_columns = ['pool_category_' + str(i) for i in range(pool_category_encoded.shape[1])]

#Concatenating one hot-encoded and other features to same dataframe
df_encoded = pd.concat([df_encoded, pd.DataFrame(pool_category_encoded, columns=pool_category_columns)], axis=1)

df_encoded = df_encoded.drop(columns = ['pool', 'pool_category'])

#One value in the pool size columns couldn't be calculated, it's replaced with the mean which shouldn't bias results
df_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
df_encoded.fillna(df_encoded.mean(), inplace=True)

#Standardizing the features
scaler = StandardScaler()
X = scaler.fit_transform(df_encoded.drop(columns=['return_in_1month']))
y = df_encoded['return_in_1month']

#Defining a train and validation split. The test data can later be used to test the best model
#Split is same as before
train_size = int(0.75 * len(X))  
test_size = len(X) - train_size

#Preparing train and test data
train_X, train_y = X[:train_size], y[:train_size]
test_X, test_y = X[train_size:], y[train_size:]

#Converting test features and target to pytorch tensors (might have to use them later)
test_inputs = torch.tensor(test_X, dtype=torch.float32).view(-1, X.shape[1])
test_labels = torch.tensor(test_y.values, dtype=torch.float32).view(-1, 1)

#Defining the architecture of the deep neural network
class Net(nn.Module):
    def __init__(self, dropout_rate):
        #DNN has one hidden layer, a batch normalization layer, an output layer and a dropout layer
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 64)
        self.bn1 = nn.BatchNorm1d(64) 
        self.fc2 = nn.Linear(64, 1)  
        self.dropout = nn.Dropout(dropout_rate)  

    def forward(self, x):
        #activation function is relu, others have been tried out but didn't perform better
        #defining how the layers are arranged
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

#setting the num of folds for time-series cross validation
tscv = TimeSeriesSplit(n_splits=6)

def cross_val_score(lr: float, weight_decay: float, dropout_rate: float):
    batch_size = 30
    model = Net(dropout_rate)
    #Different optimizers such as adam or adagrad have been tested out. SGD performs the best
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
    
    mae_scores = []
    mse_scores = []
    rmse_scores = []
    hit_rates = []
    
    #Iterating over each time-series split
    for train_index, val_index in tscv.split(train_X):
        #Splitting up the data into training and validation features + labels
        train_inputs, train_labels = torch.tensor(train_X[train_index], dtype=torch.float32), torch.tensor(train_y.iloc[train_index].values, dtype=torch.float32).view(-1, 1)
        val_inputs, val_labels = torch.tensor(train_X[val_index], dtype=torch.float32), torch.tensor(train_y.iloc[val_index].values, dtype=torch.float32).view(-1, 1)

        #Converting training and validation data to tensor
        train_dataset = TensorDataset(train_inputs, train_labels)
        trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        val_dataset = TensorDataset(val_inputs, val_labels)
        valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


        #Training the model
        for epoch in range(250):
            model.train()
            for inputs, labels in trainloader:
                optimizer.zero_grad()
                output = model(inputs)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()

            #Validating the model
            model.eval()
            with torch.no_grad():
                all_preds, all_labels = [], []
                for inputs, labels in valloader:
                    output = model(inputs)
                    all_preds.extend(output.view(-1).tolist())
                    all_labels.extend(labels.view(-1).tolist())

        mse_scores.append(mean_squared_error(all_labels, all_preds))
        rmse_scores.append(np.sqrt(mean_squared_error(all_labels, all_preds)))
        mae_scores.append(mean_absolute_error(all_labels, all_preds)) 
        hit_rates.append(np.mean((np.array(all_preds) > 0) == (np.array(all_labels) > 0)))
    
    print(f'Mean MSE: {np.mean(mse_scores)}')
    print(f'Mean RMSE: {np.mean(rmse_scores)}')
    print(f'Mean MAE: {np.mean(mae_scores)}')
    print(f'Mean Hit rate: {np.mean(hit_rates)}')
    
    return -np.mean(mae_scores)  

#Defining parameter grid
bounds = {
    'lr': (0.00001, 0.01),
    'weight_decay': (0.000001, 0.001),
    'dropout_rate': (0.1, 0.5)
}

#Optimizing the hyperparameter search with bayesian optimization
optimizer = BayesianOptimization(
    f=lambda lr, weight_decay, dropout_rate: cross_val_score(lr, weight_decay, dropout_rate),
    pbounds=bounds,
    random_state=63,
)

optimizer.maximize(
    init_points=3,
    n_iter=10,
)


print(optimizer.max)


In [ ]:
#Three with Bayesian Optimization (SGD) and time-series cross-validation
#Encoding "pool" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder = OneHotEncoder(sparse_output=False)
pool_encoded = encoder.fit_transform(df_final[['pool']])
pool_columns = ['pool_' + str(i) for i in range(pool_encoded.shape[1])]

#Concatenating one hot-encoded and other features to same dataframe
df_encoded = pd.concat([pd.DataFrame(pool_encoded, columns=pool_columns), df_final.reset_index(drop=True)], axis=1)

#Encoding "pool_category" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder_pool_category = OneHotEncoder(sparse_output=False)  
pool_category_encoded = encoder_pool_category.fit_transform(df_final[['pool_category']])
pool_category_columns = ['pool_category_' + str(i) for i in range(pool_category_encoded.shape[1])]

#Concatenating one hot-encoded and other features to same dataframe
df_encoded = pd.concat([df_encoded, pd.DataFrame(pool_category_encoded, columns=pool_category_columns)], axis=1)

df_encoded = df_encoded.drop(columns = ['pool', 'pool_category'])

#One value in the pool size columns couldn't be calculated, it's replaced with the mean which shouldn't bias results
df_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
df_encoded.fillna(df_encoded.mean(), inplace=True)

#Standardizing the features
scaler = StandardScaler()
X = scaler.fit_transform(df_encoded.drop(columns=['return_in_1month']))
y = df_encoded['return_in_1month']

#Defining a train and validation split. The test data can later be used to test the best model
#Split is same as before
train_size = int(0.75 * len(X))  
test_size = len(X) - train_size

#Preparing train and test data
train_X, train_y = X[:train_size], y[:train_size]
test_X, test_y = X[train_size:], y[train_size:]

#Converting test features and target to pytorch tensors (might have to use them later)
test_inputs = torch.tensor(test_X, dtype=torch.float32).view(-1, X.shape[1])
test_labels = torch.tensor(test_y.values, dtype=torch.float32).view(-1, 1)

#Defining the architecture of the deep neural network
class Net(nn.Module):
    def __init__(self, dropout_rate):
        super(Net, self).__init__()
        #DNN has three hidden layers, three batch normalization layer, an output layer and a dropout layer
        self.fc1 = nn.Linear(X.shape[1], 256)  
        self.bn1 = nn.BatchNorm1d(256) 
        self.fc2 = nn.Linear(256, 128) 
        self.bn2 = nn.BatchNorm1d(128)  
        self.fc3 = nn.Linear(128, 64)   
        self.bn3 = nn.BatchNorm1d(64)   
        self.fc4 = nn.Linear(64, 1)     
        self.dropout = nn.Dropout(dropout_rate)  

    def forward(self, x):
        #activation function is relu, others have been tried out but didn't perform better
        #defining how the layers are arranged
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

#setting the num of folds for time-series cross validation
tscv = TimeSeriesSplit(n_splits=6)

def cross_val_score(lr: float, weight_decay: float, dropout_rate: float):
    batch_size = 30
    model = Net(dropout_rate)
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
    
    mae_scores = []
    mse_scores = []
    rmse_scores = []
    hit_rates = []
    
    #Iterating over each time-series split
    for train_index, val_index in tscv.split(train_X):
        train_inputs, train_labels = torch.tensor(train_X[train_index], dtype=torch.float32), torch.tensor(train_y.iloc[train_index].values, dtype=torch.float32).view(-1, 1)
        val_inputs, val_labels = torch.tensor(train_X[val_index], dtype=torch.float32), torch.tensor(train_y.iloc[val_index].values, dtype=torch.float32).view(-1, 1)

        train_dataset = TensorDataset(train_inputs, train_labels)
        trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        val_dataset = TensorDataset(val_inputs, val_labels)
        valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


        #Training the model
        for epoch in range(250):
            model.train()
            for inputs, labels in trainloader:
                optimizer.zero_grad()
                output = model(inputs)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()

            #Validating the model
            model.eval()
            with torch.no_grad():
                all_preds, all_labels = [], []
                for inputs, labels in valloader:
                    output = model(inputs)
                    all_preds.extend(output.view(-1).tolist())
                    all_labels.extend(labels.view(-1).tolist())

        mse_scores.append(mean_squared_error(all_labels, all_preds))
        rmse_scores.append(np.sqrt(mean_squared_error(all_labels, all_preds)))
        mae_scores.append(mean_absolute_error(all_labels, all_preds)) 
        hit_rates.append(np.mean((np.array(all_preds) > 0) == (np.array(all_labels) > 0)))   
        
    print(f'Mean MSE: {np.mean(mse_scores)}')
    print(f'Mean RMSE: {np.mean(rmse_scores)}')
    print(f'Mean MAE: {np.mean(mae_scores)}')
    print(f'Mean Hit rate: {np.mean(hit_rates)}')
    
    return -np.mean(mae_scores)  

#Defining parameter grid
bounds = {
    'lr': (0.00001, 0.01),
    'weight_decay': (0.000001, 0.001),
    'dropout_rate': (0.1, 0.5)
}

#Optimizing the hyperparameter search with bayesian optimization
optimizer = BayesianOptimization(
    f=lambda lr, weight_decay, dropout_rate: cross_val_score(lr, weight_decay, dropout_rate),
    pbounds=bounds,
    random_state=63,
)

optimizer.maximize(
    init_points=3,
    n_iter=10,
)


print(optimizer.max)


In [ ]:
#Five with Bayesian Optimization (SGD) and time-series cross-validation
#Encoding "pool" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder = OneHotEncoder(sparse_output=False)
pool_encoded = encoder.fit_transform(df_final[['pool']])
pool_columns = ['pool_' + str(i) for i in range(pool_encoded.shape[1])]

#Concatenating one hot-encoded and other features to same dataframe
df_encoded = pd.concat([pd.DataFrame(pool_encoded, columns=pool_columns), df_final.reset_index(drop=True)], axis=1)

#Encoding "pool_category" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder_pool_category = OneHotEncoder(sparse_output=False)  
pool_category_encoded = encoder_pool_category.fit_transform(df_final[['pool_category']])
pool_category_columns = ['pool_category_' + str(i) for i in range(pool_category_encoded.shape[1])]

#Concatenating one hot-encoded and other features to same dataframe
df_encoded = pd.concat([df_encoded, pd.DataFrame(pool_category_encoded, columns=pool_category_columns)], axis=1)

df_encoded = df_encoded.drop(columns = ['pool', 'pool_category'])

#One value in the pool size columns couldn't be calculated, it's replaced with the mean which shouldn't bias results
df_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
df_encoded.fillna(df_encoded.mean(), inplace=True)

#Standardizing the features
scaler = StandardScaler()
X = scaler.fit_transform(df_encoded.drop(columns=['return_in_1month']))
y = df_encoded['return_in_1month']

#Defining a train and validation split. The test data can later be used to test the best model
#Split is same as before
train_size = int(0.75 * len(X))  
test_size = len(X) - train_size

#Preparing train and test data
train_X, train_y = X[:train_size], y[:train_size]
test_X, test_y = X[train_size:], y[train_size:]

#Converting test features and target to pytorch tensors (might have to use them later)
test_inputs = torch.tensor(test_X, dtype=torch.float32).view(-1, X.shape[1])
test_labels = torch.tensor(test_y.values, dtype=torch.float32).view(-1, 1)

#Defining the architecture of the deep neural network
class Net(nn.Module):
    def __init__(self, dropout_rate):
        super(Net, self).__init__()
        #DNN has five hidden layers, five batch normalization layer, an output layer and a dropout layer
        self.fc1 = nn.Linear(X.shape[1], 256)  
        self.bn1 = nn.BatchNorm1d(256)  
        self.fc2 = nn.Linear(256, 128)  
        self.bn2 = nn.BatchNorm1d(128)  
        self.fc3 = nn.Linear(128, 64)  
        self.bn3 = nn.BatchNorm1d(64)  
        self.fc4 = nn.Linear(64, 32)  
        self.bn4 = nn.BatchNorm1d(32)  
        self.fc5 = nn.Linear(32, 16)  
        self.bn5 = nn.BatchNorm1d(16)  
        self.fc6 = nn.Linear(16, 1) 
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        #activation function is relu, others have been tried out but didn't perform better
        #defining how the layers are arranged
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = F.relu(self.bn4(self.fc4(x)))
        x = self.dropout(x)
        x = F.relu(self.bn5(self.fc5(x)))
        x = self.dropout(x)
        x = self.fc6(x)
        return x

#setting the num of folds for time-series cross validation
tscv = TimeSeriesSplit(n_splits=6)

def cross_val_score(lr: float, weight_decay: float, dropout_rate: float):
    batch_size = 30
    model = Net(dropout_rate)
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
    
    mae_scores = []
    mse_scores = []
    rmse_scores = []
    hit_rates = []
    
    #Iterating over each time-series split
    for train_index, val_index in tscv.split(train_X):
        train_inputs, train_labels = torch.tensor(train_X[train_index], dtype=torch.float32), torch.tensor(train_y.iloc[train_index].values, dtype=torch.float32).view(-1, 1)
        val_inputs, val_labels = torch.tensor(train_X[val_index], dtype=torch.float32), torch.tensor(train_y.iloc[val_index].values, dtype=torch.float32).view(-1, 1)

        train_dataset = TensorDataset(train_inputs, train_labels)
        trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        val_dataset = TensorDataset(val_inputs, val_labels)
        valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


        #Training the model
        for epoch in range(250):
            model.train()
            for inputs, labels in trainloader:
                optimizer.zero_grad()
                output = model(inputs)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()

            #Validating the model
            model.eval()
            with torch.no_grad():
                all_preds, all_labels = [], []
                for inputs, labels in valloader:
                    output = model(inputs)
                    all_preds.extend(output.view(-1).tolist())
                    all_labels.extend(labels.view(-1).tolist())

        mse_scores.append(mean_squared_error(all_labels, all_preds))
        rmse_scores.append(np.sqrt(mean_squared_error(all_labels, all_preds)))
        mae_scores.append(mean_absolute_error(all_labels, all_preds)) 
        hit_rates.append(np.mean((np.array(all_preds) > 0) == (np.array(all_labels) > 0)))   
        
    print(f'Mean MSE: {np.mean(mse_scores)}')
    print(f'Mean RMSE: {np.mean(rmse_scores)}')
    print(f'Mean MAE: {np.mean(mae_scores)}')
    print(f'Mean Hit rate: {np.mean(hit_rates)}')
    
    return -np.mean(mae_scores)  

#Defining parameter grid
bounds = {
    'lr': (0.00001, 0.01),
    'weight_decay': (0.000001, 0.001),
    'dropout_rate': (0.1, 0.5)
}

#Optimizing the hyperparameter search with bayesian optimization
optimizer = BayesianOptimization(
    f=lambda lr, weight_decay, dropout_rate: cross_val_score(lr, weight_decay, dropout_rate),
    pbounds=bounds,
    random_state=63,
)

optimizer.maximize(
    init_points=3,
    n_iter=10,
)


print(optimizer.max)


## GRU model

In [ ]:
#Preparing data for recurrent neural networks
df_no_missing = df_final_RNN.dropna()

#Encoding "pool" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder = OneHotEncoder(sparse_output=False)
pool_encoded = encoder.fit_transform(df_no_missing[['pool']])
pool_columns = ['pool_' + str(i) for i in range(pool_encoded.shape[1])]

#Concatenating one hot-encoded and other features to same dataframe
df_encoded = pd.concat([pd.DataFrame(pool_encoded, columns=pool_columns), df_no_missing.reset_index(drop=True)], axis=1)

#Encoding "pool_category" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder_pool_category = OneHotEncoder(sparse_output=False)
pool_category_encoded = encoder_pool_category.fit_transform(df_no_missing[['pool_category']])
pool_category_columns = ['pool_category_' + str(i) for i in range(pool_category_encoded.shape[1])]

#Concatenating one hot-encoded and other features to same dataframe
df_encoded = pd.concat([df_encoded, pd.DataFrame(pool_category_encoded, columns=pool_category_columns)], axis=1)

df_encoded = df_encoded.drop(columns = ['pool_category'])

#One value in the pool size columns couldn't be calculated, it's replaced with the mean which shouldn't bias results
df_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
df_encoded['total_USD_value'].fillna(df_encoded['total_USD_value'].mean(), inplace=True)

#Dropping unnecessary columns
df_encoded = df_encoded.drop(columns = ["platform", "end_date", "symbol_1", "symbol_2"])


In [ ]:
#Trying out a GRU model
#Code to create sequences from the input data
def create_sequences(input_data, labels, seq_length):
    sequences = []
    seq_labels = []

    for i in range(len(input_data) - seq_length + 1):
        sequence = input_data[i:i+seq_length].values  
        sequence = torch.tensor(sequence, dtype=torch.float32)  
        sequences.append(sequence)
        seq_labels.append(labels[i+seq_length-1])

    return torch.stack(sequences), torch.tensor(seq_labels, dtype=torch.float32).view(-1, 1)

#Split data into train and test sets (75% of data is used for training, 25% for testing)
train_size = int(0.75 * len(df_encoded))
test_size = len(df_encoded) - train_size

train_data = df_encoded[:train_size]
test_data = df_encoded[train_size:]

train_data = train_data.sort_values(by=['pool', 'start_date'])
test_data = test_data.sort_values(by=['pool', 'start_date'])

train_data = train_data.drop(columns = ['pool', 'start_date'])
test_data = test_data.drop(columns = ['pool', 'start_date'])

#The longer the more historic info taken into account but also the more complex training
seq_length = 2

#Bring input data for each pool into the rigth format (there's 35 pools with 24 observations each)
train_sequences = []
train_labels = []
for i in range(35):
    pool_df = train_data[i*18:(i+1)*18].reset_index(drop=True)
    pool_inputs, pool_labels = create_sequences(pool_df.drop(columns=['return_in_1month']), 
                                                pool_df['return_in_1month'], seq_length)
    train_sequences.append(pool_inputs)
    train_labels.append(pool_labels)
    
seq_length = 1

#Same for test data
test_sequences = []
test_labels = []
for i in range(35):
    pool_df = test_data[i*6:(i+1)*6].reset_index(drop=True)
    pool_inputs, pool_labels = create_sequences(pool_df.drop(columns=['return_in_1month']), 
                                                pool_df['return_in_1month'], seq_length)
    test_sequences.append(pool_inputs)
    test_labels.append(pool_labels)

#Converting rrain_sequences and test_sequences to lists of tensors
train_sequences = torch.cat(train_sequences)
train_labels = torch.cat(train_labels)
test_sequences = torch.cat(test_sequences)
test_labels = torch.cat(test_labels)

#Converting training and validation data to tensor form
train_data = TensorDataset(train_sequences, train_labels)
test_data = TensorDataset(test_sequences, test_labels)

#Provides model with batches of training and validation data 
#No shuffling to preserve time series nature of data
trainloader = DataLoader(train_data, batch_size=30, shuffle=False)
testloader = DataLoader(test_data, batch_size=30, shuffle=False)

#Defining model architecture
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout_rate):
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.gru(x, h0)  
        out = self.fc(out[:, -1, :])  
        return out

#Setting up time-series cross val
tscv = TimeSeriesSplit(n_splits=6)

def train_model(lr: float, weight_decay: float, hidden_dim: int, n_layers: int, dropout_rate: float):
    #To return the best model
    global best_model
    hidden_dim = int(hidden_dim)
    n_layers = int(n_layers)
    model = Net(input_dim=train_sequences.shape[2], hidden_dim=hidden_dim, n_layers=n_layers, dropout_rate=dropout_rate)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.1)

    best_score = np.inf
    
    best_model = None

    mse_scores, rmse_scores, mae_scores, hit_rates = [], [], [], []

    for train_index, val_index in tscv.split(train_sequences):
        train_split = train_sequences[train_index]
        val_split = train_sequences[val_index]

        train_split_labels = train_labels[train_index]
        val_split_labels = train_labels[val_index]

        train_data = TensorDataset(train_split, train_split_labels)
        val_data = TensorDataset(val_split, val_split_labels)

        trainloader = DataLoader(train_data, batch_size=30, shuffle=False)
        valloader = DataLoader(val_data, batch_size=30, shuffle=False)

        model.train()
        epochs = 1000
        best_loss = np.inf
        patience = 20
        patience_counter = 0

        for epoch in range(epochs):
            running_loss = 0.0
            for inputs, labels in trainloader:
                optimizer.zero_grad()
                output = model(inputs)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            model.eval()
            val_running_loss = 0.0
            with torch.no_grad():
                for inputs, labels in valloader:
                    output = model(inputs)
                    loss = criterion(output, labels)
                    val_running_loss += loss.item()

                    mse = mean_squared_error(labels.detach().numpy(), output.detach().numpy())
                    rmse = np.sqrt(mse)
                    mae = mean_absolute_error(labels.detach().numpy(), output.detach().numpy())
                    hit_rate = np.mean((np.sign(labels.detach().numpy()) == np.sign(output.detach().numpy())))

                    mse_scores.append(mse)
                    rmse_scores.append(rmse)
                    mae_scores.append(mae)
                    hit_rates.append(hit_rate)

            scheduler.step(val_running_loss)

            if val_running_loss < best_loss:
                best_loss = val_running_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    break

        if best_loss < best_score:
            best_score = best_loss
            best_model = copy.deepcopy(model)

    print(f'Mean MSE: {np.mean(mse_scores)}')
    print(f'Mean RMSE: {np.mean(rmse_scores)}')
    print(f'Mean MAE: {np.mean(mae_scores)}')
    print(f'Mean Hit rate: {np.mean(hit_rates)}')

    return -best_score

#Defining grid for hyperparam search with bayesian opt
bounds = {
    'lr': (0.00001, 0.01),
    'weight_decay': (0.000001, 0.001),
    'hidden_dim': (50, 500),
    'n_layers': (1, 3),
    'dropout_rate': (0.1, 0.5)
}

#Defining bayesian optimizer
optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=63,
)

#Optimizer runs 25 times, changing three params each
optimizer.maximize(
    init_points=15,
    n_iter= 1
)

print(optimizer.max)  


## LSTM model

In [ ]:
#Trying out a LSTM model
#Code to create sequences from the input data
def create_sequences(input_data, labels, seq_length):
    sequences = []
    seq_labels = []

    for i in range(len(input_data) - seq_length + 1):
        sequences.append(input_data[i:i+seq_length])  
        seq_labels.append(labels[i+seq_length-1])  

    return torch.stack(sequences), torch.tensor(seq_labels, dtype=torch.float32).view(-1, 1)

#The longer the more historic info taken into account but also the more complex training
seq_length = 6

#Bring input data for each pool into the rigth format (there's 35 pools with 8 observations each)
pool_data = []
for i in range(35):
    #Pool_df contains observations on each pool
    pool_df = df_encoded[i*8:(i+1)*8]
    #Extracting features
    pool_inputs = torch.tensor(pool_df.drop(columns=['return_in_1month']).values, dtype=torch.float32)
    #Extracting target
    pool_labels = torch.tensor(pool_df['return_in_1month'].values, dtype=torch.float32)
    #Creating the sequences of features and target
    pool_sequences, pool_seq_labels = create_sequences(pool_inputs, pool_labels, seq_length)
    pool_data.append((pool_sequences, pool_seq_labels))

#Concat data from all pools
inputs = torch.cat([data[0] for data in pool_data])
labels = torch.cat([data[1] for data in pool_data])

#Split data into train and validation sets (75% of data is used for training, 25% for validation)
#We would also need to define a test set but performance is very far off from other models, which means we wouldn't use this model anywaystrain_size = int(0.75 * len(inputs))
train_sequences, train_labels = inputs[:train_size], labels[:train_size]
val_sequences, val_labels = inputs[train_size:], labels[train_size:]

#Converting training and validation data to tensor form
train_data = TensorDataset(train_sequences, train_labels)
val_data = TensorDataset(val_sequences, val_labels)

#Provides model with batches of training and validation data 
#No shuffling to preserve time series nature of data
trainloader = DataLoader(train_data, batch_size=30, shuffle=False)
valloader = DataLoader(val_data, batch_size=30, shuffle=False)

#Defining model architecture
class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout_rate):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def train_model(lr, weight_decay, hidden_dim, n_layers, dropout_rate):
    hidden_dim = int(hidden_dim)
    n_layers = int(n_layers)
    model = LSTMNet(input_dim=train_sequences.shape[2], hidden_dim=hidden_dim, n_layers=n_layers, dropout_rate=dropout_rate)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.1)

    #Training the model
    epochs = 1000
    best_loss = np.inf
    best_model = None
    patience = 20
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        val_running_loss = 0.0
        with torch.no_grad():
            for inputs, labels in valloader:
                output = model(inputs)
                loss = criterion(output, labels)
                val_running_loss += loss.item()

        scheduler.step(val_running_loss)

        #Early stopping if no further progress is being made in training
        if val_running_loss < best_loss:
            best_loss = val_running_loss
            best_model = copy.deepcopy(model)
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping on epoch {epoch}')
                break
    #negative loss is metric we want to maximize (want to minimize loss)
    return -best_loss

#Defining grid for hyperparam search with bayesian opt
bounds = {
    'lr': (1e-5, 1e-2),
    'weight_decay': (1e-6, 1e-3),
    'hidden_dim': (32, 128),
    'n_layers': (1, 5),
    'dropout_rate': (0.0, 0.7)
}

#Defining bayesian optimizer
optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
    verbose=2
)

#Optimizer runs 50 times, changing five params each
optimizer.maximize(init_points=5, n_iter=50)


## Re-training the best model

In [ ]:
#Model with Gradient Boosting with grid search and bayesian optimization
#Best parameters with hyperparam search are found based on train / validation set and performance of best model is then evaluated on test set
#This is being done as gradient boosting has best performance on training / validation and is therefore best model overall
df_no_missing = df_final.dropna()

#df_no_missing = df_no_missing.drop(columns = ["platform", "symbol_1", "symbol_2", "start_date", "end_date"])

#Defining target and features
y = df_no_missing['return_in_1month']
X = df_no_missing.drop(columns=['pool', 'pool_category'])

#Encoding "pool" and "pool_category" as one-hot (adding new columns and adding 1 or 0 in the corresponding rows)
encoder_pool = OneHotEncoder(sparse_output=False)
pool_encoded = encoder_pool.fit_transform(df_no_missing[['pool']])
pool_columns = ['pool_' + str(i) for i in range(pool_encoded.shape[1])]

encoder_pool_category = OneHotEncoder(sparse_output=False)
pool_category_encoded = encoder_pool_category.fit_transform(df_no_missing[['pool_category']])
pool_category_columns = ['pool_category_' + str(i) for i in range(pool_category_encoded.shape[1])]

#Concatenating one hot-encoded and other features to same dataframe
df_encoded = pd.concat([
    pd.DataFrame(pool_encoded, columns=pool_columns),
    pd.DataFrame(pool_category_encoded, columns=pool_category_columns),
    X.reset_index(drop=True)
], axis=1)

#One value in the pool size columns couldn't be calculated, it's replaced with the mean which shouldn't bias results
df_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
df_encoded.fillna(df_encoded.mean(), inplace=True)

X = df_encoded.drop(columns=['return_in_1month'])

#Initializing the gradient boosting model, adding regularization and early stopping to avoid overfitting the model
model = GradientBoostingRegressor(min_samples_split = 10, min_samples_leaf = 4, 
                                  random_state=63, max_depth=3, max_features='sqrt', 
                                  subsample=0.8, n_iter_no_change=5, tol=0.01)

#Defining parameter grid (number of trees, depth of each tree and learning rate)
param_grid = {
    'n_estimators': (1, 100),   
    'max_depth': (1, 5),          
    'learning_rate': (0.001, 0.1)   
}

#Splitting data into train / validating and test set
#Train and validation sets are used to find the best model, test set can later be used to test the best model
#According to this 4 3-month periods are used for training, 2 for validation and 2 for testing
train_size = 0.5
val_size = 0.25
test_size = 0.25

#Calculating sizes based on the proportions
num_samples = len(df_encoded)
train_samples = int(train_size * num_samples)
val_samples = int(val_size * num_samples)
test_samples = num_samples - train_samples - val_samples

X_train = df_encoded[:train_samples]
X_val = df_encoded[train_samples:train_samples + val_samples]
X_test = df_encoded[train_samples + val_samples:]
y_train = y[:train_samples]
y_val = y[train_samples:train_samples + val_samples]
y_test = y[train_samples + val_samples:]

#Concatenating train and validation dfs
X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])

#Performing time-series cross-validation to account for time-series nature of data
#75% of the total data is in the train / validation set, this corresponds to 18 observations per pool (which explains the split)
#Each fold is being used once as the validation set and the rest as the training set
tscv = TimeSeriesSplit(n_splits=18)

#Using bayes optimization to find the best hyperparam combination for my model based on the grid defined earlier
bayes_search = BayesSearchCV(model, param_grid, scoring='neg_mean_absolute_error', cv=tscv, n_jobs=-1, n_iter=50)
bayes_search.fit(X_trainval, y_trainval)

#Finding the best model (maximizing negative MAE (=minimizing MAE))
best_model = bayes_search.best_estimator_

print("Best Parameters:", bayes_search.best_params_)
print("Best Score (Negative MAE):", bayes_search.best_score_)

mse_list = []
rmse_list = []
mae_list = []
hit_rate_list = []

fold_num = 1

window_length = 6

#Fitting the previously found best model to the training data and evaluating it on the validation data
#Using a k-fold cross validation approach
for train_index, val_index in tscv.split(X_trainval):
    X_train, X_val = X_trainval.iloc[train_index], X_trainval.iloc[val_index]
    y_train, y_val = y_trainval.iloc[train_index], y_trainval.iloc[val_index]
    
    if len(X_train) >= window_length:
        best_model.fit(X_train, y_train)
        y_pred_val = best_model.predict(X_val)

        #Calculating evaluation metrics on the validation set
        mse_val = mean_squared_error(y_val, y_pred_val)
        mse_list.append(mse_val)
        rmse_val = math.sqrt(mse_val)
        rmse_list.append(rmse_val)
        mae_val = mean_absolute_error(y_val, y_pred_val)
        mae_list.append(mae_val)
        hit_rate_val = np.mean((y_val > 0) == (y_pred_val > 0))
        hit_rate_list.append(hit_rate_val)
        fold_num += 1

print(f"Average MSE on Validation Set: {np.mean(mse_list)}")
print(f"Average RMSE on Validation Set: {np.mean(rmse_list)}")
print(f"Average MAE on Validation Set: {np.mean(mae_list)}")
print(f"Average Hit Rate on Validation Set: {np.mean(hit_rate_list)}")

plt.plot(range(1, fold_num), mae_list, marker='o')
plt.xlabel('Fold')
plt.ylabel('Mean Absolute Error')
plt.title('Mean Absolute Error for Each Fold')
plt.show()

actual_values = []
predicted_values = []
lower_values = []
upper_values = [] 

#Splitting data into train_val and test sets
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=test_samples, shuffle=False)

X_test = X_test.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

X_test_copy = X_test.copy()

#Adding a period column to the test set to apply a expanding window approach
for i in range(pool_encoded.shape[1]):
    X_test_copy['pool_'+str(i)] = X_test_copy['pool_'+str(i)].cumsum()

X_test_copy['period'] = X_test_copy[['pool_' + str(i) for i in range(pool_encoded.shape[1])]].max(axis=1)

#Training two additional models to predict the 5th and 95th percentiles (to predict uncertainty)
model_lower = GradientBoostingRegressor(loss='quantile', alpha=0.05, **bayes_search.best_params_)
model_upper = GradientBoostingRegressor(loss='quantile', alpha=0.95, **bayes_search.best_params_)

#Fitting best model on train_val set
best_model.fit(X_trainval, y_trainval)
model_lower.fit(X_trainval, y_trainval)
model_upper.fit(X_trainval, y_trainval)

periods = X_test_copy['period'].unique()

#Applying walk forward approach (first predicting the first test period, then adding it to training set, refitting the model
#and predicting the second period)
for i in range(6):
    X_test_copy_period = X_test_copy[X_test_copy['period'] == periods[i]]
    X_test_period = X_test.loc[X_test_copy_period.index]
    y_test_period = y_test.loc[X_test_copy_period.index]
    y_test_period = y_test_period.reset_index(drop = True)
    X_test_copy_period = X_test_copy_period.drop('period', axis=1)

    #Predicting target variable in test set
    y_pred_test = best_model.predict(X_test_period)
    y_pred_lower = model_lower.predict(X_test_period)
    y_pred_upper = model_upper.predict(X_test_period)

    #Storing actual values, predicted values and the upper and lower bounds of the forecast
    actual_values.extend(y_test_period)
    predicted_values.extend(y_pred_test)
    lower_values.extend(y_pred_lower)  
    upper_values.extend(y_pred_upper)

    #Calculating eval metrics on test set
    mse_test = mean_squared_error(y_test_period, y_pred_test)
    rmse_test = math.sqrt(mse_test)
    mae_test = mean_absolute_error(y_test_period, y_pred_test)
    hit_rate_test = np.mean((y_test_period > 0) == (y_pred_test > 0))

    print(f"Period {periods[i]}:")
    print(f"MSE on Test Set: {mse_test}")
    print(f"RMSE on Test Set: {rmse_test}")
    print(f"MAE on Test Set: {mae_test}")
    print(f"Hit Rate on Test Set: {hit_rate_test}")

    #Adding data of the current testing period to the trainval set
    X_trainval = pd.concat([X_trainval, X_test_period])
    y_trainval = pd.concat([y_trainval, y_test_period])

    #Refitting model on new training set
    best_model.fit(X_trainval, y_trainval)
    model_lower.fit(X_trainval, y_trainval)
    model_upper.fit(X_trainval, y_trainval)

In [ ]:
#Plotting 95% confidence interval around predictions
fig, ax = plt.subplots(figsize=(12,6))

#Lineplot for actual and predicted values
ax.plot(range(len(actual_values)), actual_values, color='blue', label='Actual Return')
ax.plot(range(len(predicted_values)), predicted_values, color='red', label='Predicted Return')

#Plotting the confidence interval around predictions
ax.fill_between(range(len(predicted_values)), lower_values, upper_values, color='grey', alpha=.5, label='Confidence Interval')

ax.set_title("Predicted vs Actual Returns with a 90% Confidence Interval")
ax.set_xlabel("Index")
ax.set_ylabel("Return (%)")
plt.legend()
plt.show()


In [ ]:
#Get feature importances from best model
feature_importances = best_model.feature_importances_

#Get corresponding feature names
feature_names = X_trainval.columns

#Select top 10 features
sorted_indices = np.argsort(feature_importances)[::-1]
sorted_importances = feature_importances[sorted_indices]
sorted_feature_names = feature_names[sorted_indices]
top_indices = sorted_indices[:10]
top_importances = sorted_importances[:10]
top_feature_names = sorted_feature_names[:10]

plt.figure(figsize=(10, 6))
plt.bar(range(len(top_importances)), top_importances, tick_label=top_feature_names)
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('Importance Score')
plt.title('Top 10 Feature Importances')
plt.tight_layout()
plt.show()

In [ ]:
#Writing a function to decode pool information
def decode_pool(encoded_pool_row):
    pool_index = np.argmax(encoded_pool_row)
    return f'Pool {pool_index}'

#Defining pool colmns as they occur in df_encoded
pool_columns = [f'pool_{i}' for i in range(35)]

#Adding pool info to df
test_dataset_df = pd.DataFrame(testset[:][0].numpy(), columns=df_encoded.columns.drop('return_in_1month'))
test_dataset_df['pool_info'] = test_dataset_df[pool_columns].apply(decode_pool, axis=1)

#Adding actual and predicted return to df
test_dataset_df['actual_return'] = test_labels
test_dataset_df['predicted_return'] = test_preds

#Plotting actual vs predicted values for each pool
unique_pools = test_dataset_df['pool_info'].unique()
for pool in unique_pools:
    pool_data = test_dataset_df[test_dataset_df['pool_info'] == pool]
    
    plt.figure(figsize=(10, 5))
    plt.plot(pool_data['actual_return'].values, label='Actual')
    plt.plot(pool_data['predicted_return'].values, label='Predicted')
    plt.xlabel('Index')
    plt.ylabel('Return in 3 Months')
    plt.title(f'Returns for {pool}')
    plt.legend()
    plt.show()

## Economic inituition / optimization

In [ ]:
#Storing the addresses of the pools
pool_names = df_final["pool"].unique()

#List times six (six forecasting periods)
pool_names = np.repeat(pool_names, 6)

#Creating a period list (first 35 observations are in the the first period, ...)
periods_list = [i // 35 + 1 for i in range(210)]

#Adding all the relevant lists to a dataframe
df_results = pd.DataFrame({
    'Pool': pool_names,
    'Period': periods_list,
    'Actual_Return': actual_values,
    'Predicted_Return': predicted_values,
    'Lower bound': lower_values,
    'Upper bound': upper_values
})

#Grouping by periods and finding the pool with the highest predicted return
for period, group in df_results.groupby('Period'):
    max_pred_return_pool = group.loc[group['Predicted_Return'].idxmax(), 'Pool']
    max_pred_return = group.loc[group['Predicted_Return'].idxmax(), 'Predicted_Return']
    actual_return = group.loc[group['Predicted_Return'].idxmax(), 'Actual_Return']
    
    print(f"Period {period}:")
    print(f"Pool with highest predicted return: {max_pred_return_pool}")
    print(f"Predicted Return: {max_pred_return}")
    print(f"Actual Return: {actual_return}")

In [ ]:
#This is the case where uncertainty is not taken into account and the LP just provides liquidity for the pair with the highest return
#Assuming a uniform distribution of gas fees of 7 USD for both Uniswap and Sushiswap (double-checked with on-chain transactions)
#For adding liquidity the first time the LP only accounts the 7 USD once
#For every consequent reshuffling the LP incorporates the fee for both withdrawing and adding new liquidity (two times 7 USD)
#I arbitrarily chose three different initial sizes of liquidity positions (5'000, 50'000 and 500'000), which would approximately represent a small, large and medium LP

gas_fee = 7  
initial_position_sizes = [5000, 50000, 500000]  
position_sizes = {size: [] for size in initial_position_sizes} 

#Storing important values for plotting later
return_data = {size: {"actual": [], "predicted": [], "upper": [], "lower": []} for size in initial_position_sizes}

#Looping through every investment size
for size in initial_position_sizes:
    current_size = size
    #Looping through each period and selecting the pool with the highest return
    for period, group in df_results.groupby('Period'):
        #Pool with highest forecasted returns
        max_pred_return_pool = group.loc[group['Predicted_Return'].idxmax(), 'Pool']
        #Predicted return of this pool
        max_pred_return = group.loc[group['Predicted_Return'].idxmax(), 'Predicted_Return'] / 100
        #Actual return of this pool
        actual_return = group.loc[group['Predicted_Return'].idxmax(), 'Actual_Return'] / 100  
        
        #For plotting later
        return_data[size]["actual"].append(actual_return*100)
        return_data[size]["predicted"].append(max_pred_return*100)
        return_data[size]["upper"].append(group.loc[group['Predicted_Return'].idxmax(), 'Upper bound'])
        return_data[size]["lower"].append(group.loc[group['Predicted_Return'].idxmax(), 'Lower bound'])
        
        #For the first period, only the cost of adding liquidity is incorporated
        if period == 1:
            liquidity_cost = gas_fee
        #For every subsequent period also the cost of withdrawing the liquidity is added
        else:
            liquidity_cost = gas_fee * 2

        #If predicted return in % * current_size > cost of adding (+withdrawing) liquidity, the LP choses to add liquidity / switch to a new pool
        if max_pred_return * current_size > liquidity_cost:
            #Assuming that the new positions depends on the actual return in the period and the cost of switching the pair
            new_position_size = current_size * (1 + actual_return) - liquidity_cost 
            position_sizes[size].append(new_position_size)
        else:
            #If pair is not switched then the LP just gets the return of the pool in that period
            actual_return_noswitch = df_final_RNN.loc[(df_final_RNN['start_date'] == period) & (df_final_RNN['pool'] == max_pred_return_pool), 'return'] / 100
            #The size of the position after the period is just influenced by the return of the pool
            new_position_size = current_size * (1 + actual_return_noswitch)
            position_sizes[size].append(new_position_size)

        current_size = new_position_size  


for size, positions in position_sizes.items():
    print(f"Final position size for initial size {size}: {positions[-1]}")


In [ ]:
#Plotting actual and predicted values and confidence intervals around them
date_mapping = {i: start_date + pd.DateOffset(months=i-1) for i in df_results['Period'].unique()}

for size in return_data:
    df = pd.DataFrame({
        'actual_values': return_data[size]['actual'],
        'predicted_values': return_data[size]['predicted'],
        'upper_bound': return_data[size]['upper'],
        'lower_bound': return_data[size]['lower']
    }, index=[date_mapping[i] for i in range(1, len(return_data[size]['actual']) + 1)])

    plt.figure(figsize=(14, 7))
    plt.plot(df['actual_values'], color='blue', label='Actual Values')
    plt.plot(df['predicted_values'], color='red', label='Predicted Values')
    plt.fill_between(df.index, df['lower_bound'], df['upper_bound'], color='gray', alpha=0.2, label='Confidence Interval')
    plt.title(f'Returns for Initial Position', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Return (%)', fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
#Second optimization strategy: Sharpe Ratio is minimized
#First period in forecasting set
start_date = pd.to_datetime('2022-09-01')

#Zero is assumed for simplicity but could also be the return for staking in a crypto context
risk_free_rate = 0

#Average gas fee for liquidity provision (both DEX and withdrawals + deposits) according to on-chain data
gas_fee = 7

#Gas fee might render certain position sizes unprofitable (not really the case with small gas fee)
initial_position_sizes = [5000, 50000, 500000]
position_sizes = {size: [] for size in initial_position_sizes}

#Storing important values for plotting later
return_data = {size: {"actual": [], "predicted": [], "upper": [], "lower": []} for size in initial_position_sizes}

#To keep track of the pool we're currently invested in
current_pool = {}

#Looping through investment sizes
for size in initial_position_sizes:
    current_size = size
    position_sizes[size] = []
    current_pool[size] = None

    #Looping through periods and pools
    for period, group in df_results.groupby('Period'):
        #Calculate Sharpe Ratio for each pool
        #Note that the formula for volatility assumes symmetry around the mean / normal distribution, this is most likely not the case but is good enough for this approach
        group['Volatility'] = (group['Upper bound'] - group['Lower bound']) / (2 * 1.96)
        group['Excess_Return'] = group['Predicted_Return'] - risk_free_rate
        group['Sharpe_Ratio'] = group['Excess_Return'] / group['Volatility']

        #Selecting the pool with the highest Sharpe ratio
        optimal_pool_data = group.loc[group['Sharpe_Ratio'].idxmax()]
        
        #For plotting later              
        return_data[size]["actual"].append(optimal_pool_data['Actual_Return'])
        return_data[size]["predicted"].append(optimal_pool_data['Predicted_Return'])
        return_data[size]["upper"].append(optimal_pool_data['Upper bound'])
        return_data[size]["lower"].append(optimal_pool_data['Lower bound'])
        
        #Pool with highest Sharpe ratio
        max_sharpe_ratio_pool = optimal_pool_data['Pool']
        #Expected return of that pool
        expected_return = optimal_pool_data['Predicted_Return'] / 100
        actual_return = optimal_pool_data['Actual_Return'] / 100

        liquidity_cost = 0
        
        #Only incur liquidity cost if there's a pool switch (gas fees paid twice) or it's the first period (gas fees paid once)
        if period == 1 or max_sharpe_ratio_pool != current_pool[size]:
            liquidity_cost = gas_fee if period == 1 else gas_fee * 2

        #if expected return of switching larger than cost, then switch
        if expected_return * current_size > liquidity_cost:
            actual_return_noswitch = actual_return
            current_pool[size] = max_sharpe_ratio_pool 
    
        #if not larger then stay and take return of current pool
        else:
            actual_return_noswitch = df_final_RNN.loc[(df_final_RNN['start_date'] == date_mapping[period]) & (df_final_RNN['pool'] == current_pool[size]), 'return'].values[0] / 100
            print(f"Period {date_mapping[period]}: No liquidity provision due to high gas fees. Using actual return of {actual_return_noswitch*100}% for calculations.")
            
        #The size of the position after the period is influenced by the return and the cost of switching the pool
        #If pool is not switched then liquidity_cost = 0
        new_position_size = current_size * (1 + actual_return_noswitch) - liquidity_cost 
        position_sizes[size].append(new_position_size)

        current_size = new_position_size

for size, positions in position_sizes.items():
    print(f"Final position size for initial size {size}: {positions[-1]}")

In [ ]:
#Plotting actual values, predicted values and the confidence interval around them
for size in return_data:
    df = pd.DataFrame({
        'actual_values': return_data[size]['actual'],
        'predicted_values': return_data[size]['predicted'],
        'upper_bound': return_data[size]['upper'],
        'lower_bound': return_data[size]['lower']
    }, index=[date_mapping[i] for i in range(1, len(return_data[size]['actual']) + 1)])

    plt.figure(figsize=(14, 7))
    plt.plot(df['actual_values'], color='blue', label='Actual Values')
    plt.plot(df['predicted_values'], color='red', label='Predicted Values')
    plt.fill_between(df.index, df['lower_bound'], df['upper_bound'], color='gray', alpha=0.2, label='Confidence Interval')
    plt.title(f'Returns for Initial Position', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Return (%)', fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.show()

## Alternative baseline strategies

In [ ]:
#Function to determine the top pools by either size, volume or average return or lowest avg. vola
def select_top_pools(df, date, n, metric, lookback_months=None, return_based=False, volatility_based=False):
    #Consider whole timeframe for calculations
    if lookback_months is None:
        mask = (df['start_date'] < date)
    #Only consider last n months
    else:
        lookback_date = date - pd.DateOffset(months=lookback_months)
        mask = (df['start_date'] < date) & (df['start_date'] >= lookback_date)
        
    #For return mean of last n returns is taken and then top n pools
    if return_based:
        top_n_pools = df.loc[mask].groupby('pool')[metric].mean().nlargest(n).index.tolist()
    #For vola mean of last n 30d_volas is taken and then top n pools with smallest
    elif volatility_based:
        top_n_pools = df.loc[mask].groupby('pool')[metric].mean().nsmallest(n).index.tolist()
    #For other two metrics sum is taken and then top n pools
    else:
        top_n_pools = df.loc[mask].groupby('pool')[metric].sum().nlargest(n).index.tolist()

    return top_n_pools

#As before, three investment sizes
initial_investments = [5000, 50000, 500000]

#Defining the strategy pool (four metrics, two timehorizons, two num_pools -> 16 metrics)
strategies = [
    {"lookback_months": 6, "num_pools": 1, "metric": "total_USD_value"},
    {"lookback_months": 6, "num_pools": 5, "metric": "total_USD_value"},
    {"lookback_months": None, "num_pools": 1, "metric": "total_USD_value"},
    {"lookback_months": None, "num_pools": 5, "metric": "total_USD_value"},
    {"lookback_months": 6, "num_pools": 1, "metric": "volume_usd"},
    {"lookback_months": 6, "num_pools": 5, "metric": "volume_usd"},
    {"lookback_months": None, "num_pools": 1, "metric": "volume_usd"},
    {"lookback_months": None, "num_pools": 5, "metric": "volume_usd"},
    {"lookback_months": 6, "num_pools": 1, "metric": "return", "return_based": True},
    {"lookback_months": 6, "num_pools": 5, "metric": "return", "return_based": True},
    {"lookback_months": None, "num_pools": 1, "metric": "return", "return_based": True},
    {"lookback_months": None, "num_pools": 5, "metric": "return", "return_based": True},
    {"lookback_months": 6, "num_pools": 1, "metric": "30D_volatility", "volatility_based": True},
    {"lookback_months": 6, "num_pools": 5, "metric": "30D_volatility", "volatility_based": True},
    {"lookback_months": None, "num_pools": 1, "metric": "30D_volatility", "volatility_based": True},
    {"lookback_months": None, "num_pools": 5, "metric": "30D_volatility", "volatility_based": True}
]

#Timeframe under consideration
start_date = pd.to_datetime('2022-09-01')
end_date = start_date + pd.DateOffset(months=5)

#Assumed gas fee as before
gas_fee = 7

#Dictionary to store the amounts after each period for each investment amount and strategy (for plotting)
amounts_after_each_period = {amount: {str(strategy): [] for strategy in strategies} for amount in initial_investments}

#Looping through investment amounts
for amount in initial_investments:
    #Looping through strategies
    for strategy in strategies:
        current_amount = amount
        first_time_investment = True
        previous_pools = []
        pool_investments = {pool: current_amount/len(previous_pools) if previous_pools else current_amount for pool in previous_pools}
        #Looping through each month
        for month in pd.date_range(start=start_date, end=end_date, freq='MS'):
            #Selecting the top n pools
            top_pools = select_top_pools(df_final_RNN, month, strategy['num_pools'], strategy['metric'], strategy['lookback_months'])
            
            #For the first investment period, the gas fees are only deducted once
            #Also the amount invested in each pool is total amount / number of pools
            if first_time_investment:
                current_amount -= gas_fee * len(top_pools)
                current_amount = current_amount / len(top_pools) if top_pools else current_amount
                first_time_investment = False
                
            #For every subsequent period, it is checked whether pools have been switched
            else:
                switched_pools = set(previous_pools).symmetric_difference(set(top_pools))

                #if pools have been switched then the amount currently invested in that pool is stored in current_amount and corrected with the gas fees for deopist + withdrawal
                #Furthermore that entry is removed from the dict tracking the investments
                for pool in switched_pools:
                    if pool in pool_investments:
                        current_amount = pool_investments[pool]
                        current_amount -= gas_fee * 2
                        del pool_investments[pool]
            #The current amount is then attributed to the pool that has been switched into (which is currently in top_pools but not in pool_investments)            
            for pool in top_pools:
                if pool not in pool_investments: 
                    pool_investments[pool] = current_amount
            
            current_amount = 0
            #For all pools that are in top_pools in a period their actual returns in that period are looked up and their returns are compounded
            for pool in top_pools:
                mask = (df_final_RNN['start_date'] == month) & (df_final_RNN['pool'] == pool)
                pool_return = df_final_RNN.loc[mask, 'return'].values[0] / 100
                pool_investments[pool] *= (1 + pool_return)

            previous_pools = top_pools.copy()

            #Calculates the total value of the investments at the end of all periods
            current_amount = sum(pool_investments.values())
            #Stores the investment amounts at the end of each period for each strategy
            amounts_after_each_period[amount][str(strategy)].append(current_amount)

        print(f"Return after six months (Lookback: {strategy['lookback_months']}, Num Pools: {strategy['num_pools']}, Metric: {strategy['metric']}): {current_amount}")

In [ ]:
#Renaming the strategies for plotting
strategy_names = {
    str({'lookback_months': 6, 'num_pools': 1, 'metric': 'total_USD_value'}): "Pool size - Last six months- 1 pool",
    str({'lookback_months': 6, 'num_pools': 5, 'metric': 'total_USD_value'}): "Pool size - Last six months - 5 pools",
    str({'lookback_months': None, 'num_pools': 1, 'metric': 'total_USD_value'}): "Pool size - All time - 1 pool",
    str({'lookback_months': None, 'num_pools': 5, 'metric': 'total_USD_value'}): "Pool size - All time - 5 pools",
    str({'lookback_months': 6, 'num_pools': 1, 'metric': 'volume_usd'}): "Volume - Last six months- 1 pool",
    str({'lookback_months': 6, 'num_pools': 5, 'metric': 'volume_usd'}): "Volume - Last six months - 5 pools",
    str({'lookback_months': None, 'num_pools': 1, 'metric': 'volume_usd'}): "Volume - All time - 1 pool",
    str({'lookback_months': None, 'num_pools': 5, 'metric': 'volume_usd'}): "Volume - All time - 5 pools",
    str({'lookback_months': 6, 'num_pools': 1, 'metric': 'return', 'return_based': True}): "Avg. return - Last six months- 1 pool",
    str({'lookback_months': 6, 'num_pools': 5, 'metric': 'return', 'return_based': True}): "Avg. return - Last six months - 5 pools",
    str({'lookback_months': None, 'num_pools': 1, 'metric': 'return', 'return_based': True}): "Avg. return - All time - 1 pool",
    str({'lookback_months': None, 'num_pools': 5, 'metric': 'return', 'return_based': True}): "Avg. return - All time - 5 pools",
    str({'lookback_months': 6, 'num_pools': 1, 'metric': '30D_volatility', 'volatility_based': True}): "Avg. vola - Last six months- 1 pool",
    str({'lookback_months': 6, 'num_pools': 5, 'metric': '30D_volatility', 'volatility_based': True}): "Avg. vola - Last six months - 5 pools",
    str({'lookback_months': None, 'num_pools': 1, 'metric': '30D_volatility', 'volatility_based': True}): "Avg. vola - All time - 1 pool",
    str({'lookback_months': None, 'num_pools': 5, 'metric': '30D_volatility', 'volatility_based': True}): "Avg. vola - All time - 5 pools",
}

#Retrieving the final investment amounts for each strategy
final_returns = {amount: {str(strategy): amounts[-1] if amounts else 0 for strategy, amounts in strategy_amounts.items()} for amount, strategy_amounts in amounts_after_each_period.items()}

#Returns of the more profitable optimization (optimization by expected returns)
custom_returns = [0.01, -0.001, -0.108, 0.164, 0.234, 0.035]

custom_returns_2 = [-0.13, -0.001, -0.024, 0.164, 0.305, 0.056]

def calculate_custom_amounts(amount, returns):
    amounts = []
    current_amount = amount
    for return_ in returns:
        current_amount *= (1 + return_)
        amounts.append(current_amount)
    return amounts

def plot_strategies(strategies, title):
    plt.figure(figsize=(15, 6))
    for strategy in strategies:
        plt.plot(pd.date_range(start=start_date, end=end_date, freq='MS'), strategy_amounts[strategy], label=strategy_names.get(strategy, strategy))
    plt.title(title)
    plt.xlabel("Month")
    plt.ylabel("Invested Amount")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.show()

#Calculating custom returns
custom_return_sets = [custom_returns, custom_returns_2]
custom_return_names = ["Custom: Predicted return", "Custom: Risk-adjusted return"]

for amount in initial_investments:
    for returns, name in zip(custom_return_sets, custom_return_names):
        amounts_after_each_period[amount][name] = calculate_custom_amounts(amount, returns)

#Building final_returns
final_returns = {}
for amount, strategy_amounts in amounts_after_each_period.items():
    final_returns[amount] = {}
    for strategy, amounts in strategy_amounts.items():
        final_returns[amount][str(strategy)] = amounts[-1] if amounts else 0

#Plotting
for amount, strategy_amounts in amounts_after_each_period.items():
    #Plot all strategies
    plot_strategies(strategy_amounts.keys(), "All Strategies")
    
    #Selecting top five strategies by return
    top_strategies = sorted(final_returns[amount].items(), key=lambda item: item[1], reverse=True)[:5]

    #Plotting top five strategies
    plot_strategies([strategy for strategy, _ in top_strategies], f"Top 5 Strategies")
